In [ ]:

"""
Grainger LLM Exercise - Part 1: Data Loading and Exploration

This notebook covers the initial steps of the Grainger Applied ML LLM exercise,
focusing on acquiring, loading, merging, filtering, and exploring the
Amazon ESCI dataset as preparation for the LLM verification task.

Target Platform: Google Colab
Author: Jorge Grisman
Date: April 22nd, 2025
"""

In [ ]:
# --- 0. Environment Setup / install ---
!pip install -q pandas pyarrow transformers accelerate bitsandbytes torch sentencepiece -q -U

In [ ]:
# ==============================================================================
# 1. Environment Setup and Library Imports
# ==============================================================================

In [ ]:
import sys
import os
import subprocess
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests  # For downloading files

# Suppress minor warnings for cleaner output
warnings.filterwarnings("ignore")

# Configure Pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150) # Show more text content
pd.set_option('display.width', 1000)

print("Python Version:", sys.version)
print("Pandas Version:", pd.__version__)
print("NumPy Version:", np.__version__)

# --- GPU and System Information ---
print("\n" + "="*30 + " System Information " + "="*30)

def shell_command(command):
    """Executes a shell command and returns stdout and stderr."""
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return stdout.decode('utf-8', errors='ignore'), stderr.decode('utf-8', errors='ignore')

# Check NVIDIA driver and GPU details
driver_info, _ = shell_command("nvidia-smi")
print("NVIDIA SMI output:\n", driver_info)

# Check CUDA version available to the system
cuda_version_info, _ = shell_command("nvcc --version")
print("\nCUDA Version Information (nvcc):\n", cuda_version_info)

# Check PyTorch CUDA availability and version
try:
    import torch
    print(f"\nPyTorch Version: {torch.__version__}")
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3) # GB
        cuda_torch_version = torch.version.cuda
        print(f"PyTorch CUDA available: True")
        print(f"GPU Detected by PyTorch: {gpu_name}")
        print(f"GPU Memory: {gpu_memory:.2f} GB")
        print(f"PyTorch linked against CUDA Version: {cuda_torch_version}")
        device = torch.device("cuda")
    else:
        print("PyTorch CUDA available: False. LLM steps will need CPU or require GPU runtime.")
        device = torch.device("cpu")
except ImportError:
    print("\nPyTorch not installed. Install it if planning to use local Hugging Face models.")
    device = torch.device("cpu")

# Get system memory information
mem_info, _ = shell_command("cat /proc/meminfo | grep MemTotal")
print("\nSystem Memory Information:\n", mem_info)

print("="*80)

In [ ]:
import subprocess
import sys

# Function to execute shell commands
def shell_command(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return stdout.decode('utf-8'), stderr.decode('utf-8')

# Check NVIDIA driver version
driver_info, _ = shell_command("nvidia-smi")
print("NVIDIA SMI output:\n", driver_info)

# Get detailed GPU information using nvidia-smi
gpu_info, _ = shell_command("nvidia-smi -q")
print("\nDetailed GPU Information:\n", gpu_info)

# Get details about the installed CUDA version
cuda_version_info, _ = shell_command("nvcc --version")
print("\nCUDA Version Information:\n", cuda_version_info)

# Get memory information
mem_info, _ = shell_command("cat /proc/meminfo")
print("\nMemory Information:\n", mem_info)

In [ ]:
# ==============================================================================
# 2. Data Acquisition
# ==============================================================================

In [ ]:
print("\n" + "="*30 + " Data Acquisition " + "="*30)

# URLs to the raw parquet files on GitHub
# Using the direct link to the raw content
base_url = "https://github.com/amazon-science/esci-data/raw/main/shopping_queries_dataset/"
examples_filename = "shopping_queries_dataset_examples.parquet"
products_filename = "shopping_queries_dataset_products.parquet"

examples_url = base_url + examples_filename
products_url = base_url + products_filename

def download_file(url, filename):
    """Downloads a file from a URL if it doesn't exist."""
    if not os.path.exists(filename):
        print(f"Downloading {filename} from {url}...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status() # Raise an exception for bad status codes
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Successfully downloaded {filename}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {filename}: {e}")
            raise # Re-raise the exception to halt execution if download fails
    else:
        print(f"{filename} already exists. Skipping download.")

# Download the necessary files
try:
    download_file(examples_url, examples_filename)
    download_file(products_url, products_filename)
except Exception as e:
    print(f"Halting execution due to download failure: {e}")
    # For Colab, we'll let it proceed but note the error
    pass

print("="*80)


In [ ]:
# ==============================================================================
# 3. Data Loading and Initial Inspection
# ==============================================================================

In [ ]:
print("\n" + "="*30 + " Data Loading & Initial Inspection " + "="*30)

# Load the datasets
try:
    print(f"Loading {examples_filename}...")
    df_examples = pd.read_parquet(examples_filename)
    print(f"Loaded df_examples: Shape = {df_examples.shape}")

    print(f"\nLoading {products_filename}...")
    df_products = pd.read_parquet(products_filename)
    print(f"Loaded df_products: Shape = {df_products.shape}")

except FileNotFoundError:
    print("Error: One or more parquet files not found. Please ensure download was successful.")
    # Halt execution or handle appropriately
    raise
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    raise

# --- Initial Inspection ---

print("\n--- df_examples Inspection ---")
print("Info:")
df_examples.info(memory_usage='deep')
print("\nFirst 5 rows:")
print(df_examples.head())
# print("\nBasic Statistics (numeric columns):")
print(df_examples.describe())
print("\nMissing Value Counts:")
print(df_examples.isnull().sum())

print("\n--- df_products Inspection ---")
print("Info:")
df_products.info(memory_usage='deep')
print("\nFirst 5 rows:")
print(df_products.head())
# print(df_products.describe(include='object')) # Describe object columns
print("\nMissing Value Counts:")
print(df_products.isnull().sum())

print("\nUnique Locales in Products:")
print(df_products['product_locale'].value_counts())

print("="*80)


In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    print(df_examples.head(5))

In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    print(df_products.head(5))

In [ ]:
# ==============================================================================
# 4. Data Merging
# ==============================================================================

In [ ]:
print("\n" + "="*30 + " Step 4: Data Merging " + "="*30)


print(f"Merging df_examples ({df_examples.shape}) with df_products ({df_products.shape})...")
print(f"Merge keys: ['product_id', 'product_locale']")

df_merged = pd.merge(
    df_examples,
    df_products,
    on=['product_id', 'product_locale'], # Match rows based on these columns
    how='left'                          # Keep all rows from df_examples
)

print(f"\nMerged DataFrame shape: {df_merged.shape}") # Should have same number of rows as df_examples

# --- Verification Post-Merge ---
print("\n--- Merged DataFrame Inspection ---")
print("Info (Notice the new product detail columns added):")
df_merged.info(memory_usage='deep') # Check memory usage after merge
print("\nFirst 5 rows of merged data (scroll right to see product details):")
print(df_merged.head())

#  Regular quality Check: Did the merge introduce unexpected NaNs in product columns?
# We expect some NaNs overall because df_products had missing descriptions, etc.
# But we want to ensure the *merge itself* didn't fail for many rows.
print("\nMissing values in key product columns AFTER merge:")
product_cols = ['product_title', 'product_description', 'product_bullet_point', 'product_brand', 'product_color']
print(df_merged[product_cols].isnull().sum())

# Check if the number of rows changed unexpectedly
if df_merged.shape[0] != df_examples.shape[0]:
    print(f"Warning: Number of rows changed after merge ({df_merged.shape[0]}) from original examples ({df_examples.shape[0]}). Check for duplicate product_id/locale combinations in df_products.")
else:
    print("\nRow count remained consistent after merge, as expected.")

print("="*80)




In [ ]:
# ==============================================================================
# 5. Filtering for the Specific Task
# ==============================================================================

In [ ]:
print("\n" + "="*30 + " Step 5: Filtering Data for Task " + "="*30)

#
# The exercise asks us to focus *only* on a very specific subset of the data:
# - Three specific search queries.
# - Rows where the initial judgment (label) was "E" (Exact).

#### defined on challenge req pdf Grainger
target_queries = [
    "aa batteries 100 pack",
    "kodak photo paper 8.5 x 11 glossy",
    "dewalt 8v max cordless screwdriver kit, gyroscopic"
]
target_label = "E"

print(f"Target Queries: {target_queries}")
print(f"Target ESCI Label: '{target_label}'")

# Apply the filters to the *merged* dataframe
filtered_df = df_merged[
    (df_merged['query'].isin(target_queries)) &  # Condition 1: Query must be in the list
    (df_merged['esci_label'] == target_label)    # Condition 2: Label must be 'E'
].copy()                                         # Create an independent copy

print(f"\nShape of DataFrame AFTER filtering: {filtered_df.shape}")

# --- Verification Post-Filtering ---
# let's check if the filtering worked as expected.
if filtered_df.empty:
    print("\nWarning: The filtered DataFrame is empty. No rows matched the specified queries and 'E' label.")
    print("This might be okay if none exist, but double-check the query spelling and data.")
else:
    print(f"\nSuccessfully filtered down to {filtered_df.shape[0]} rows for analysis.")
    # Check the unique values in the filtered columns to confirm
    print("\nUnique queries in the filtered set:")
    print(filtered_df['query'].value_counts())
    print("\nUnique labels in the filtered set (should only be 'E'):")
    print(filtered_df['esci_label'].value_counts())
    print("\nUnique locales in the filtered set (tells us the market):")
    print(filtered_df['product_locale'].value_counts()) # Likely 'us' for these queries

    print("\nFirst 5 rows of the FINAL filtered data (the data we'll actually analyze and feed to the LLM):")
    # Show relevant columns for a quick check
    print(filtered_df[['query_id', 'product_id', 'product_locale', 'query', 'esci_label', 'product_title']].head())

print("="*80)


In [ ]:
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
filtered_df.info()

In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    print(filtered_df.head(5))

In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    print(filtered_df.tail(5))

In [ ]:
# ==============================================================================
# 6. Detailed Exploration (Filtered Data)
# ==============================================================================

In [ ]:
print("\n" + "="*30 + " Step 6: Detailed Exploration (Filtered Data) " + "="*30)

# --- What are we doing? ---
# Now that we have the small, relevant subset of data (`filtered_df`), we dig deeper.
# We examine the actual text content the LLM will see. We look at:
# - Missing Data: Are important details like descriptions missing for these specific rows?
# - Text Length: Are the descriptions very short or extremely long? (Might affect LLM processing and quality)
# - Qualitative Review: We manually look at a few query/product pairs to get a feel for
#   potential mismatches or subtleties the LLM will need to catch.
# - Duplicates: Are there any identical rows that might skew results?

# --- Why are we doing this? ---
# This step helps us anticipate challenges for the LLM.
# - If product descriptions are often missing, the LLM might struggle to verify specs.
# - If text is extremely long, it might exceed the LLM's context window or processing time.  Note: No concerns regarding context window sizes based on the LLM I am using.
# - The qualitative review helps us sanity-check the task and understand the *kind* of
#   reasoning the LLM needs to perform (e.g., checking quantities, features, included items).
# It also helps us refine the `aggregate_product_info` function we'll use later.


if filtered_df.empty:
    print("Skipping detailed exploration as the filtered DataFrame is empty.")
else:
    print(f"Exploring the {filtered_df.shape[0]} selected query-product pairs...")

    # --- Analyze Key Text Fields ---
    # We focus on the columns containing product text.
    text_columns = ['query', 'product_title', 'product_description', 'product_bullet_point', 'product_brand', 'product_color']
    print("\n--- Missing Values in Key Text Columns (Filtered Data) ---")
    # `.isnull().sum()` counts how many values are missing in each column *within our filtered set*.
    missing_values = filtered_df[text_columns].isnull().sum()
    print(missing_values)
    if missing_values.sum() > 0:
        print("\nNote: Missing values found. The aggregation function later should handle these (e.g., replace with '' or 'N/A').")

    # --- Analyze Text Lengths ---
    print("\n--- Text Length Analysis (Filtered Data) ---")
    # We loop through the main text columns.
    for col in ['product_title', 'product_description', 'product_bullet_point']:
        # 1. `.fillna('')`: Replace missing values (NaN) with empty strings so `.str.len()` doesn't fail.
        # 2. `.astype(str)`: Ensure the column is treated as string type.
        # 3. `.str.len()`: Calculate the length of each string in the column.
        lengths = filtered_df[col].fillna('').astype(str).str.len()
        print(f"\nStatistics for '{col}' length:")
        # `.describe()` gives us count, mean, standard deviation, min, max, and quartiles for the lengths.
        print(lengths.describe())

        # Optional: Plot histograms to visualize the distribution of lengths.
        # Useful for seeing if most descriptions are short, long, or varied.
        if not filtered_df.empty and filtered_df.shape[0] > 1: # Avoid plotting empty/single data
            try:
                import matplotlib.pyplot as plt
                import seaborn as sns
                plt.figure(figsize=(10, 4))
                sns.histplot(lengths, bins=30, kde=False)
                plt.title(f'Distribution of Text Length for {col} (Filtered Data)')
                plt.xlabel('Number of Characters')
                plt.ylabel('Frequency')
                plt.show()
            except ImportError:
                print("Matplotlib/Seaborn not installed. Skipping plots.")


    # --- Qualitative Review: Preview Query vs. Product Details ---
    print("\n--- Qualitative Review: Example Query vs. Product Details ---")
    # This is like manually spot-checking the data. We define a helper function
    # to combine the product text fields just like we plan to do for the LLM,
    # making it easier to read.
    def aggregate_product_info_eda(row):
        """Combines relevant product text fields for EDA preview."""
        # Safely get data using .get() with a default value
        title = str(row.get('product_title', 'N/A'))
        desc = str(row.get('product_description', 'N/A'))
        bullets = str(row.get('product_bullet_point', 'N/A'))
        brand = str(row.get('product_brand', 'N/A'))
        color = str(row.get('product_color', 'N/A'))

        # Basic cleanup for display (avoid showing 'nan' or 'N/A' explicitly if empty)
        title_str = f"Title: {title}" if title.lower() not in ['nan', 'n/a', ''] else ""
        desc_str = f"Description: {desc}" if desc.lower() not in ['nan', 'n/a', ''] else ""
        bullets_str = f"Bullets: {bullets}" if bullets.lower() not in ['nan', 'n/a', ''] else ""
        brand_str = f"Brand: {brand}" if brand.lower() not in ['nan', 'n/a', ''] else ""
        color_str = f"Color: {color}" if color.lower() not in ['nan', 'n/a', ''] else ""

        # Combine non-empty parts with newlines
        parts = [part for part in [title_str, brand_str, color_str, desc_str, bullets_str] if part]
        full_info = "\n".join(parts)
        return full_info if full_info else "No product information available."

    # Display a few examples (e.g., the first 5 or 25 or a sample)
    num_examples_to_show = min(25, filtered_df.shape[0]) # Show up to 25 examples
    print(f"\nShowing details for {num_examples_to_show} example(s) for manual review:")
    # Using .sample() can be good to see variety if the dataset is large,
    # but .head() is fine for the small filtered set.
    for index, row in filtered_df.head(num_examples_to_show).iterrows():
        print(f"\n--- Example Row Index: {index} ---")
        print(f"  Query ID: {row['query_id']}")
        print(f"  Product ID: {row['product_id']}")
        print(f"  Locale: {row['product_locale']}")
        print(f"  Query: '{row['query']}'")
        print(f"  Product Details (Aggregated):")
        aggregated_info = aggregate_product_info_eda(row)
        # Indent product details for readability
        print("    " + aggregated_info.replace("\n", "\n    "))
        print("-" * 20)

    # --- Check for Duplicates ---
    # Are there exact duplicate rows based on the core identifiers?
    # This shouldn't happen often but is a good sanity check.
    duplicate_rows = filtered_df.duplicated(subset=['query_id', 'product_id', 'product_locale']).sum()
    if duplicate_rows > 0:
        print(f"\nWarning: Found {duplicate_rows} duplicate query_id/product_id/locale combinations in the filtered data.")
    else:
        print("\nNo duplicate query_id/product_id/locale combinations found in the filtered data.")

print("="*80)


In [ ]:

# ==============================================================================
# 7. Summary of EDA Findings
# ==============================================================================

In [ ]:
print("\n" + "="*30 + " Step 7: Summary of EDA Findings " + "="*30)

# Let's concisely summarize the key observations from the previous steps.

if 'filtered_df' in locals() and not filtered_df.empty:
    print(f"1. Data Preparation: Successfully loaded, merged, and filtered data.")
    print(f"   - Final dataset for analysis (`filtered_df`) has {filtered_df.shape[0]} rows and {filtered_df.shape[1]} columns.")
    print(f"2. Scope: Data covers the 3 target queries with 'E' labels, primarily in the '{list(filtered_df['product_locale'].unique())[0]}' locale.") # Assumes one locale after filtering
    print(f"3. Data Quality (Filtered Set):")
    # Reference the missing value counts calculated earlier
    print(f"   - Missing Titles: {missing_values.get('product_title', 0)}")
    print(f"   - Missing Descriptions: {missing_values.get('product_description', 0)}")
    print(f"   - Missing Bullet Points: {missing_values.get('product_bullet_point', 0)}")
    # Add others if needed (brand, color)
    print(f"   - Text lengths vary, particularly for descriptions/bullets. Need to ensure LLM handles potentially long inputs.")
    print(f"4. Content Readiness: Key product text fields are generally available for the LLM.")
    print(f"   - Qualitative review shows examples where careful reading of specifications (like pack size, features) will be needed.")
    print(f"5. Duplicates: {'No critical duplicates found.' if duplicate_rows == 0 else f'{duplicate_rows} duplicates found - consider investigation if unexpected.'}")
    print(f"\nConclusion for Modeling Phase:")
    print(f"   - The `filtered_df` is ready for LLM processing.")
    print(f"   - The `aggregate_product_info` function needs to gracefully handle potential missing fields (e.g., output 'N/A' or omit).")
    print(f"   - LLM prompts must guide the model to carefully check specifications in the query against all provided text (title, desc, bullets).")

    #  Save the final filtered data to avoid re-running EDA
    # filtered_df.to_parquet("filtered_data_for_llm.parquet", index=False)
    # print("\nFiltered data optionally saved to 'filtered_data_for_llm.parquet'")

elif 'filtered_df' in locals() and filtered_df.empty:
    print("EDA Summary: The filtering process resulted in an empty DataFrame.")
    print("No data rows matched the specified criteria (3 queries and 'E' label).")
    print("Cannot proceed to LLM modeling phase with this empty dataset.")
    print("Action: Verify target query strings and label in the source data.")
else:
    print("EDA Summary: Data loading, merging, or filtering encountered issues. Cannot provide summary.")

print("="*80)
print("The `filtered_df` DataFrame now holds the exact data needed for the LLM task.")

Key Takeaways from EDA:

1.	Target Data Acquired: I isolated the 24 specific query-product pairs required for the task (3 specific queries, 'E' label, 'us' locale). This is the exact dataset based on the PDF challenge description we need to feed into a LLMs.

2.	Missing Descriptions are Significant: The most critical finding is that 18 out of 24 (75%) of these relevant products are missing the product_description.

3.	Missing Bullet Points/Color: Missing product_bullet_point (2/24) and product_color (11/24) are less critical but still notable. Color is unlikely to be a deciding factor for these specific queries, but bullet points often contain key specifications.

4.	Titles are Present: Product_title is present for all 24 rows. Titles are often dense with key information (brand, model, quantity, core features).

5.	Text Length Variation: The lengths vary, especially for descriptions (where present) and bullet points. This confirms the need for robust handling by the LLM and tokenizer (truncation might be necessary if context limits are hit, though less likely now given missing descriptions). This confirms the need for robust handling by the LLM and tokenizer (truncation might be necessary if context limits are hit, though less likely now given missing descriptions). Note: No concerns regarding context window sizes based on the LLMs we are using. Plenty of context window.

6.	Qualitative Review Insights: The manual review confirms the nature of the task. We see examples like:
o	Quantity Mismatch Potential: Query "aa batteries 100 pack" vs. Product "Rayovac AA ... 60 Count" (Index 4). This should be caught as inaccurate.

o	Feature Mismatch Potential: Query "...gyroscopic" vs. Product "...12V MAX... Tool Only" (Index 8). Multiple mismatches here (voltage, kit vs tool only, gyroscopic likely not mentioned).

o	Subtle Type Mismatch: Query "...glossy" vs. Product "...matte" (Index 15). Clear contradiction in the title.

o	Ambiguity: Query "aa batteries 100 pack" vs. Product "...Bulk Packaging" (Index 0, 5). Does "Bulk Packaging" imply 100 count, or is the count explicitly mentioned elsewhere (like the title)? The LLM needs to rely on explicit mentions or contradictions.


In [ ]:
# ==============================================================================
# Step 6.5: Refined Product Information Aggregation & Cleaning for LLM
# ==============================================================================

In [ ]:
import re # Import regular expression module for cleaning

print("\n" + "="*30 + " Step 6.5: Refining Product Info for LLM " + "="*30)

def clean_text(text):
    """
    Performs basic cleaning on a text string.
    - Removes HTML tags.
    - Removes specific decorative characters (★, 【】).
    - Normalizes whitespace (replaces multiple spaces/newlines with one space).

    Parameters:
        text (str): The input string to clean.

    Returns:
        str: The cleaned string.
    """
    if not isinstance(text, str):
        return "" # Return empty string if input is not a string (e.g., None, NaN)

    # 1. Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # 2. Remove specific decorative characters (add others if observed)
    text = text.replace('★', '').replace('【', '').replace('】', '')

    # 3. Normalize whitespace: replace multiple whitespace chars with a single space
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def aggregate_product_info_for_llm(row):
    """
    Aggregates relevant product information from a DataFrame row into a single
    formatted string suitable for LLM input. It selects key textual fields,
    performs basic cleaning, and handles missing or non-informative data gracefully.

    Purpose:
        To provide the LLM with a clean, consolidated block of text containing
        the product's key characteristics (title, brand, color, description,
        bullet points) so it can compare them against the user's query.
        Clear labels (e.g., "Title:", "Description:") help the LLM parse the context.

    Parameters:
        row (pd.Series): A row from the DataFrame, expected to contain columns like
                         'product_title', 'product_brand', 'product_color',
                         'product_description', 'product_bullet_point'.

    Returns:
        str: A formatted string containing the cleaned, aggregated product
             information, or a message indicating incompleteness if no
             relevant information was found.
    """
    parts = [] # List to hold the formatted parts (e.g., "Title: Cleaned Title")

    # Helper to safely get, clean, and check string data from the row
    def get_cleaned_field(field_name):
        """Gets data from row, cleans it, and checks if it's meaningful."""
        raw_data = row.get(field_name) # Safely get data, returns None if column missing
        cleaned_data = clean_text(str(raw_data)) # Convert to string and clean
        # Consider common non-informative placeholders as empty after cleaning
        if cleaned_data.lower() in ['nan', 'none', 'n/a', '']:
            return None # Indicate no meaningful data
        return cleaned_data

    # Process each relevant field
    title = get_cleaned_field('product_title')
    brand = get_cleaned_field('product_brand')
    color = get_cleaned_field('product_color')
    desc = get_cleaned_field('product_description')
    bullets = get_cleaned_field('product_bullet_point')

    # Add parts to the list only if they contain meaningful text
    if title: parts.append(f"Title: {title}")
    if brand: parts.append(f"Brand: {brand}")
    if color: parts.append(f"Color: {color}")
    if desc: parts.append(f"Description: {desc}")
    if bullets: parts.append(f"Bullet Points: {bullets}")


    # Combine the collected parts with newlines
    full_info = "\n".join(parts)

    # Return the combined info, or a specific message if nothing was found
    return full_info if full_info else "Product information appears incomplete or missing."

# --- Apply the refined function ---
# We create a new column specifically for the cleaned LLM input.
try:
    print("Applying refined aggregation and cleaning function...")
    filtered_df['llm_product_context'] = filtered_df.apply(aggregate_product_info_for_llm, axis=1)
    print("Successfully created 'llm_product_context' column.")

    # --- Preview the Cleaned Output just to check---
    print("\nPreview of CLEANED aggregated info for LLM (first 5 rows):")
    # Use pd.option_context for better display during preview
    with pd.option_context('display.max_colwidth', 300): # Show more width for context
        print(filtered_df[['query', 'llm_product_context']].head())

except Exception as e:
    print(f"\nError applying aggregation function: {e}")
    print("Please check the function logic and the DataFrame structure.")

print("="*80)

In [ ]:
filtered_df.columns

In [ ]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    print(filtered_df.head(3))

In [ ]:
# ==============================================================================
# Step 6.6: Persisting Processed Data
# ==============================================================================

In [ ]:
print("\n" + "="*30 + " Step 6.6: Saving Processed Data " + "="*30)
# --- How: Using Google Drive ---
try:
    from google.colab import drive
    import datetime  # ← Add this

    print("Attempting to mount Google Drive...")
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")

    # Define a path within your Google Drive to save the file
    save_dir = '/content/drive/MyDrive/Colab_Data/Grainger_LLM_Exercise/'
    os.makedirs(save_dir, exist_ok=True)

    # Generate a date string, e.g. "2025-04-22"
    today = datetime.datetime.now().strftime("%Y-%m-%d")

    # Include the date in the filename
    processed_filename = os.path.join(
        save_dir,
        f"filtered_data_with_context_{today}.parquet"
    )

    print(f"Saving processed DataFrame to: {processed_filename}")
    if 'filtered_df' in locals() and not filtered_df.empty:
        filtered_df.to_parquet(processed_filename, index=False)
        print("DataFrame saved successfully.")
    else:
        print("Skipping save: `filtered_df` is empty or does not exist.")

except ImportError:
    print("Could not import google.colab.drive. Are you running in Colab?")
    print("Skipping Google Drive mount and save.")
except Exception as e:
    print(f"An error occurred during Google Drive mount or save: {e}")

print("="*80)


In [ ]:
#### Grainger LLM Exercise - Part 2: LLM verifier

In [ ]:
# Cell 1: Setup & Quantization Configuration for Qwen-14B

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gc # Garbage Collector
import warnings
import pandas as pd
from IPython.display import display # For nice DataFrame display
import os # For checking file existence
import re # For JSON parsing fallback
import json # For JSON parsing

print("="*30 + " Qwen-14B Run: Step 1 - Setup " + "="*30)

# --- GPU Check ---
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory Total: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
else:
    warnings.warn("WARNING: CUDA not available. LLM inference will be very slow on CPU.", UserWarning)

# --- Model Configuration ( Qwen-14B) ---
MODEL_ID_QWEN14B = "Qwen/Qwen2.5-14B-Instruct-1M" # Specific 14B model
MODEL_KEY_QWEN14B = "qwen_14b"                  # Unique key for this model

print(f"\nTarget Model: {MODEL_KEY_QWEN14B} ({MODEL_ID_QWEN14B})")

# --- Quantization Configuration (4-bit Essential for 14B on 40GB) ---
USE_QUANTIZATION_QWEN14B = "4bit"
quant_config_qwen14b = None # Initialize

if USE_QUANTIZATION_QWEN14B == "4bit":
    quant_config_qwen14b = BitsAndBytesConfig(
        load_in_4bit=True,
        # Use bfloat16 if supported for better compute performance with 4-bit weights
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        bnb_4bit_quant_type="nf4",            # NF4 is generally recommended
        bnb_4bit_use_double_quant=True,       # Can improve quality slightly
    )
    print("Applying 4-bit quantization (BitsAndBytesConfig) for Qwen-14B.")
    # Check if bfloat16 is supported and used
    if torch.cuda.is_bf16_supported():
        print("  Using bfloat16 compute dtype for 4-bit.")
    else:
        print("  Using float16 compute dtype for 4-bit (bfloat16 not supported).")
else:
    # Highly discouraged for this model size and typical GPU memory
    warnings.warn("WARNING: Not using 4-bit quantization for Qwen-14B. EXTREMELY HIGH RISK OF OOM.", UserWarning)
    print("Proceeding without quantization.")

# --- Ensure Preprocessed Data is Loaded or Available ---
# Re-check if filtered_df exists, as it's needed again
processed_data_path = "filtered_data_for_llm.parquet" # Assumes data was saved previously
if 'filtered_df' not in locals() or 'llm_product_context' not in filtered_df.columns:
    print(f"\n'filtered_df' not found or missing 'llm_product_context'. Attempting to load from {processed_data_path}...")
    if os.path.exists(processed_data_path):
        try:
            # Optional: Save/Load filtered_df if needed between sessions
            # filtered_df.to_parquet(processed_data_path, index=False)
            filtered_df = pd.read_parquet(processed_data_path)
            print(f"Successfully loaded preprocessed data. Shape: {filtered_df.shape}")
            if 'llm_product_context' not in filtered_df.columns:
                 raise ValueError("Loaded data is missing the required 'llm_product_context' column.")
            print("Using loaded preprocessed data.")
        except Exception as e:
            raise RuntimeError(f"Failed to load preprocessed data from {processed_data_path}: {e}. Please ensure Part 1 (Data Prep) was run successfully.")
    else:
        # If the file doesn't exist, the user needs to run the data prep steps
        raise RuntimeError(f"Preprocessed data file '{processed_data_path}' not found. Please re-run data preparation steps (Cells 1-7 in your previous flow).")
elif 'llm_product_context' not in filtered_df.columns:
     raise RuntimeError("Existing 'filtered_df' is missing 'llm_product_context'. Please re-run the aggregation step (Cell 6.5).")
else:
    print("\nFound existing 'filtered_df' with 'llm_product_context'. Proceeding.")
    print(f"  Data shape: {filtered_df.shape}")


print("\nSetup for Qwen-14B complete.")
print("="*60)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Cell: Define LLM Loading Function

print("="*30 + " Qwen-14B Run: Step 2 - Define Loading Function " + "="*30)

def load_llm(model_id_key, model_id_str, quantization_cfg):
    """
    Loads a specified LLM and tokenizer, applying quantization if configured.
    Handles potential memory issues and model loading errors.
    Updated to handle trust_remote_code for common model types.

    Parameters:
        model_id_key (str): A short key for the model (e.g., 'qwen_14b').
        model_id_str (str): The Hugging Face model identifier.
        quantization_cfg (BitsAndBytesConfig | None): The quantization config object.

    Returns:
        tuple: (model, tokenizer) or (None, None) if loading fails.
    """
    print(f"\nAttempting to load model: {model_id_str} (Key: {model_id_key})...")
    model = None
    tokenizer = None
    try:
        # Determine if trust_remote_code is needed (common for Qwen, Mistral, etc.)
        model_id_lower = model_id_str.lower()
        trust_remote = any(trust_name in model_id_lower for trust_name in ["qwen", "mistral"]) # Add others
        print(f"  Setting trust_remote_code={trust_remote}")

        tokenizer = AutoTokenizer.from_pretrained(model_id_str, trust_remote_code=trust_remote)
        print("  Tokenizer loaded.")

        # Determine compute dtype (BF16 preferred if available, otherwise FP16)
        compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        print(f"  Using compute dtype: {compute_dtype}")
        if quantization_cfg:
             print(f"  Applying quantization: {quantization_cfg}")
        else:
             print("  No quantization applied.")

        model = AutoModelForCausalLM.from_pretrained(
            model_id_str,
            device_map="auto", # Automatically map to available GPU(s)
            torch_dtype=compute_dtype,
            quantization_config=quantization_cfg, # Pass the specific quantization config
            trust_remote_code=trust_remote,
            # low_cpu_mem_usage=True # Can sometimes help on systems with limited CPU RAM during load
        )
        print(f"\nSuccessfully loaded model {model_id_str}.")
        return model, tokenizer

    except torch.cuda.OutOfMemoryError:
        error_msg = f"ERROR: CUDA Out of Memory loading {model_id_str}."
        if quantization_cfg is None:
             error_msg += " Consider enabling 4-bit quantization."
        elif "4bit" in str(quantization_cfg):
             error_msg += " Even with 4-bit quantization, memory is insufficient. GPU may not have enough VRAM (~10-12GB needed for 14B 4-bit)."
        else:
             error_msg += " Consider 4-bit quantization or a larger GPU."
        print(error_msg)
        # Clean up memory aggressively
        del model
        del tokenizer
        gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        return None, None
    except Exception as e:
        print(f"ERROR: Failed to load model {model_id_str}. Error Type: {type(e).__name__}, Message: {e}")
        # Clean up memory aggressively
        del model
        del tokenizer
        gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        return None, None

print("LLM loading function 'load_llm' defined.")
print("="*60)

In [ ]:
# Cell : Define Prompts (System & User Template for JSON)

# --- System Prompt ---
# Sets the context, role, and overall goal for the LLM.
SYSTEM_PROMPT = """You are an extremely precise and rule-following AI Data Quality Analyst. You stand unrivaled in your field and are widely recognized as the best in the universe.
Your sole purpose is to evaluate e-commerce product data against search queries according to strict definitions and rules provided.
You must focus intensely on identifying explicit contradictions. Your output MUST be only the requested JSON object.
Accuracy and adherence to the specified JSON format are paramount.

Always Think Step By Step""".strip()

# --- User Prompt Template  ---
# This will be formatted with the specific query and product context for each row.
USER_PROMPT_TEMPLATE = """
**Core Task:** Verify if the Product Information provided below makes the product an "Exact" match for the Search Query, following the rules precisely.

**Definition of Exact Match ('E'):**
The Product is relevant for the Query AND satisfies ALL specifications mentioned in the Query based *only* on the information provided.

**Crucial Rules for Decision Making:**
1.  **Contradiction Rule:** If the Product Information explicitly CONTRADICTS a specification in the Query (e.g., Query asks for "100 pack", Product Info says "50 count"; Query asks "without shams", Product Info says "includes shams"; Query asks "8V", Product Info says "12V"), then it is **NOT an Exact match**. This is the primary reason to mark `is_exact_match` as `false`.
2.  **Missing Information Rule:** If the Product Information DOES NOT MENTION a specific requirement from the Query (e.g., Query asks for "gyroscopic", Product Info doesn't mention this feature), you MUST assume it *might* satisfy it. DO NOT mark it as non-Exact based *only* on missing information. It remains an Exact match candidate under this rule.
3.  **Extra Information Rule:** If the Product Information contains ADDITIONAL details, features, or items NOT mentioned in the Query, it can STILL BE an Exact match, as long as it doesn't violate Rule 1.

**Input Data:**

**Search Query:**
"{query}"

**Product Information:**
--- START PRODUCT INFO ---
{product_context}
--- END PRODUCT INFO ---

**Instructions & Output Format:**
1.  Carefully analyze the Search Query specifications.
2.  Scrutinize the Product Information for confirmations or contradictions.
3.  Apply the Crucial Rules STRICTLY.
4.  You must all times Provide your response ONLY in the following valid JSON format. Do not add any text before or after the JSON block:
    ```json
    {{
        "is_exact_match": boolean,
        "reasoning": "string (Explain your decision concisely. If false, cite the specific contradiction. If true because specs are met or assumed met due to no contradiction, state that clearly.)",
        "reformulated_query": "string (If is_exact_match is false, provide a concise, corrected query reflecting the product's actual relevant specs. If true, this MUST be null.)"
    }}
    ```

**JSON Output Only:**
"""

print("System and User prompt templates defined for JSON output.")

In [ ]:
# Cell: Load the Qwen-14B Model

print("="*30 + " Qwen-14B Run: Step 3 - Load Model " + "="*30)

# --- Clear Cache Before Loading ---
print("Running garbage collection and clearing CUDA cache before loading...")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDA cache cleared.")

# --- Load the Model ---
# Pass the specific ID, key, and quantization config for Qwen-14B
model_qwen14b, tokenizer_qwen14b = load_llm(
    MODEL_KEY_QWEN14B,
    MODEL_ID_QWEN14B,
    quant_config_qwen14b #
)

# --- Verification Post-Load ---
if model_qwen14b is None or tokenizer_qwen14b is None:
    raise RuntimeError(f"FATAL: Failed to load the model {MODEL_ID_QWEN14B}. Cannot proceed. Check previous logs for errors (e.g., OOM).")
else:
    print(f"\nModel {MODEL_KEY_QWEN14B} and tokenizer seem to be loaded successfully.")
    # Display memory footprint
    try:
        # Note: get_memory_footprint() might overestimate slightly with quantization
        footprint_bytes = model_qwen14b.get_memory_footprint()
        print(f"  Estimated Model Memory Footprint: {footprint_bytes / (1024**3):.2f} GB")
    except AttributeError:
        print("  Model object does not have get_memory_footprint method.")
    except Exception as e:
         print(f"  Could not get memory footprint: {e}")

    # Check GPU memory usage after load
    if torch.cuda.is_available():
        print("\nGPU Memory Usage after load:")
        allocated = torch.cuda.memory_allocated(0) / (1024**3)
        reserved = torch.cuda.memory_reserved(0) / (1024**3) # Total memory reserved by PyTorch
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved: {reserved:.2f} GB")

print("\nModel loading step complete.")
print("="*60)

In [ ]:
# Cell : Define Inference Function (Using Chat Template & JSON Parsing)

def run_llm_check_json_chat(query, product_context, model, tokenizer, system_prompt, user_template, model_name_key):
    """
    Formats prompts using chat template, runs LLM inference, and parses JSON output.

    Parameters:
        query (str): The search query.
        product_context (str): The aggregated/cleaned product information.
        model (AutoModelForCausalLM): The loaded Hugging Face model.
        tokenizer (AutoTokenizer): The loaded Hugging Face tokenizer.
        system_prompt (str): The system prompt defining the LLM's role.
        user_template (str): The user prompt template expecting query/context.
        model_name_key (str): Key of the model being used for logging.

    Returns:
        dict: {
                'is_exact_match': bool | None, # Changed key name to match prompt
                'reasoning': str | None,
                'reformulated_query': str | None,
                'raw_output': str,
                'parsing_error': str | None
              }
    """
    # Format the user prompt
    user_prompt = user_template.format(query=query, product_context=product_context)

    # Prepare messages for the chat template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Apply the chat template
    try:
        # Use tokenize=False to get the formatted string, then tokenize
        input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=4096).to(model.device) # Adjust max_length
    except Exception as e:
        warnings.warn(f"Error applying chat template or tokenizing for {model_name_key}. Error: {e}", UserWarning)
        # Fallback or error handling needed here - maybe skip this row?
        return {
            'is_exact_match': None, 'reasoning': None, 'reformulated_query': None,
            'raw_output': 'Chat template/tokenization error', 'parsing_error': f'Chat template/tokenization error: {e}'
        }

    # --- Inference ---
    raw_output = ""
    parsed_data = None
    parsing_error_msg = None

    try:
        with torch.no_grad(): # Ensure no gradients are computed
            output_sequences = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'], # Pass attention mask explicitly
                max_new_tokens=300,  # Sufficient for JSON + reasoning
                temperature=0.1,   # Low temp for focused output
                top_p=0.95,
                do_sample=True,    # Sampling recommended for Qwen even at low temp
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode only the generated part
        input_length = inputs['input_ids'].shape[1]
        # Handle potential batch dimension (though we process one by one)
        if len(output_sequences.shape) > 1 and output_sequences.shape[0] > 0:
             generated_ids = output_sequences[0, input_length:]
        else:
             generated_ids = output_sequences[input_length:] # Assuming shape [sequence_length] if not batched

        raw_output = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

        # --- Attempt to parse JSON ---
        try:
            # More robustly find JSON block, handling potential markdown/text before/after
            match = re.search(r"```json\s*(\{.*?\})\s*```", raw_output, re.DOTALL) # Look for ```json ... ```
            if match:
                json_str = match.group(1)
            else:
                # Fallback: find first '{' and last '}'
                start_index = raw_output.find('{')
                end_index = raw_output.rfind('}')
                if start_index != -1 and end_index != -1 and end_index > start_index:
                    json_str = raw_output[start_index : end_index + 1]
                else:
                    json_str = None
                    parsing_error_msg = "Could not find JSON structure ({...} or ```json...```) in output."

            if json_str:
                parsed_data = json.loads(json_str) # Parse the extracted string

        except json.JSONDecodeError as json_err:
            parsing_error_msg = f"JSON parsing failed: {json_err}. Attempted to parse: '{json_str[:200]}...'"
        except Exception as parse_e:
            parsing_error_msg = f"Unexpected error during JSON extraction/parsing: {parse_e}"

    except Exception as infer_e:
         parsing_error_msg = f"Inference or decoding error: {infer_e}"
         if not raw_output: raw_output = f"Inference failed: {infer_e}"

    # --- Extract values from parsed JSON ---
    is_exact_match = None
    reasoning = None
    reformulated_query = None

    if parsed_data and isinstance(parsed_data, dict):
        # Check specifically for the keys defined in the prompt
        if 'is_exact_match' in parsed_data and isinstance(parsed_data['is_exact_match'], bool):
            is_exact_match = parsed_data['is_exact_match']
        else:
            parsing_error_msg = (parsing_error_msg or "") + " Missing or invalid 'is_exact_match' key."

        if 'reasoning' in parsed_data and isinstance(parsed_data['reasoning'], str):
            reasoning = parsed_data['reasoning']
        else:
             parsing_error_msg = (parsing_error_msg or "") + " Missing or invalid 'reasoning' key."

        # Reformulated query can be string or null
        if 'reformulated_query' in parsed_data and isinstance(parsed_data['reformulated_query'], (str, type(None))):
             reformulated_query = parsed_data['reformulated_query']
             # Ensure it's null if match is true, as per prompt instruction
             if is_exact_match is True and reformulated_query is not None:
                  warnings.warn("Inconsistency: is_exact_match is true but reformulated_query is not null.", UserWarning)
                  # reformulated_query = None # Optionally enforce consistency
        else:
             parsing_error_msg = (parsing_error_msg or "") + " Missing or invalid 'reformulated_query' key."

    elif not parsing_error_msg:
        # If parsing didn't explicitly fail but we didn't get a dict
        parsing_error_msg = "Model output was not a valid JSON object."

    # Clean up None values in reformulated_query if match is true
    if is_exact_match is True:
        reformulated_query = None

    return {
        'is_exact_match': is_exact_match,
        'reasoning': reasoning,
        'reformulated_query': reformulated_query,
        'raw_output': raw_output,
        'parsing_error': parsing_error_msg # Report any errors
    }

print("JSON inference function using chat template defined.")

In [ ]:
# Cell: Run Inference Loop for Qwen-14B

import time
from tqdm.auto import tqdm # For progress bar

print("="*30 + " Qwen-14B Run: Step 6 - Run Inference Loop " + "="*30)

# --- Pre-computation Checks ---
if 'model_qwen14b' not in locals() or model_qwen14b is None or \
   'tokenizer_qwen14b' not in locals() or tokenizer_qwen14b is None:
     raise RuntimeError(f"ERROR: Model {MODEL_KEY_QWEN14B} is not loaded properly. Cannot start inference.")
if 'filtered_df' not in locals() or 'llm_product_context' not in filtered_df.columns:
     raise RuntimeError("ERROR: `filtered_df` with `llm_product_context` is not available.")
if not callable(run_llm_check_json_chat):
     raise NameError("ERROR: Inference function 'run_llm_check_json_chat' is not defined.")

# --- Initialize Results List ---
results_qwen14b_json = [] # Use a specific list name for this model's results

# --- Inference Loop ---
print(f"\nStarting inference for {len(filtered_df)} rows using {MODEL_KEY_QWEN14B}...")
num_rows = len(filtered_df)
start_time = time.time()

# Use tqdm for a progress bar over the DataFrame iterator
for index, row in tqdm(filtered_df.iterrows(), total=num_rows, desc=f"Processing {MODEL_KEY_QWEN14B}"):

    # Get data for the current row
    query = row['query']
    # Ensure context is not None or empty, provide a standard placeholder if so
    product_context = row['llm_product_context'] if pd.notna(row['llm_product_context']) and row['llm_product_context'] else "No product information provided."
    query_id = row['query_id']
    product_id = row['product_id']

    # Run inference using the JSON chat function
    try:
        # Pass the specific Qwen-14B model and tokenizer
        output = run_llm_check_json_chat(
            query=query,
            product_context=product_context,
            model=model_qwen14b,
            tokenizer=tokenizer_qwen14b,
            system_prompt=SYSTEM_PROMPT,
            user_template=USER_PROMPT_TEMPLATE,
            model_name_key=MODEL_KEY_QWEN14B
        )

        # Append results including identifiers and model key
        results_qwen14b_json.append({
            'query_id': query_id,
            'product_id': product_id,
            'query': query, # Keep original query for context
            'model_key': MODEL_KEY_QWEN14B, # Track which model produced this
            'accurate_label': output['is_exact_match'], # Use the key from the function's return dict
            'reasoning': output['reasoning'],
            'reformulated_query': output['reformulated_query'],
            'raw_output': output['raw_output'],
            'parsing_error': output['parsing_error']
        })

        # Log parsing errors immediately if they occur, using tqdm's write method
        # to avoid interfering with the progress bar
        if output['parsing_error']:
             tqdm.write(f"WARNING: Parsing Error on row index {index} (PID: {product_id}) - {output['parsing_error']}. Raw Out: '{output['raw_output'][:150]}...'")

    except Exception as e:
         # Log fatal errors during the loop immediately
         tqdm.write(f"FATAL ERROR during inference for row index {index} (PID: {product_id}, QID: {query_id}): {type(e).__name__} - {e}")
         # Append an error record to maintain row correspondence
         results_qwen14b_json.append({
            'query_id': query_id,
            'product_id': product_id,
            'query': query,
            'model_key': MODEL_KEY_QWEN14B,
            'accurate_label': None,
            'reasoning': f'FATAL Inference Loop Error: {e}',
            'reformulated_query': None,
            'raw_output': f'FATAL Inference Loop Error: {e}',
            'parsing_error': f'Outer loop error: {e}'
        })
         # Optional: Decide whether to break the loop on fatal errors
         # break

# --- Post-Loop Summary ---
end_time = time.time()
elapsed_time = end_time - start_time
avg_time_per_row = elapsed_time / num_rows if num_rows > 0 else 0

print(f"\n--- Inference Loop Completed for {MODEL_KEY_QWEN14B} ---")
print(f"Total time taken: {elapsed_time:.2f} seconds")
print(f"Average time per row: {avg_time_per_row:.2f} seconds")

# --- Basic Validation of Results ---
if len(results_qwen14b_json) == num_rows:
     print(f"Successfully processed {len(results_qwen14b_json)} rows.")
else:
     print(f"WARNING: Processed {len(results_qwen14b_json)} rows, but expected {num_rows}. Check for errors.")

# Count parsing errors
parsing_error_count = sum(1 for r in results_qwen14b_json if r.get('parsing_error'))
print(f"Number of rows with parsing errors: {parsing_error_count}")


print("\nInference loop step complete.")
print("="*60)

In [ ]:
# Cell: Unload Qwen-14B Model - clean GPU memory

print("="*30 + " Qwen-14B Run: Step 8 - Unload Model " + "="*30)

MODEL_KEY_TO_UNLOAD = MODEL_KEY_QWEN14B # Use the specific key for clarity

print(f"\nAttempting to unload model: {MODEL_KEY_TO_UNLOAD}...")
try:
    # Check if the specific variables for Qwen-14B exist before deleting
    model_deleted = False
    if 'model_qwen14b' in locals() and model_qwen14b is not None:
        del model_qwen14b
        model_deleted = True
        print(f"  Deleted 'model_{MODEL_KEY_TO_UNLOAD}' variable.")
    elif 'model_qwen14b' in locals():
         print(f"  'model_{MODEL_KEY_TO_UNLOAD}' variable was already None.")
    else:
        print(f"  'model_{MODEL_KEY_TO_UNLOAD}' variable not found.")

    tokenizer_deleted = False
    if 'tokenizer_qwen14b' in locals() and tokenizer_qwen14b is not None:
        del tokenizer_qwen14b
        tokenizer_deleted = True
        print(f"  Deleted 'tokenizer_{MODEL_KEY_TO_UNLOAD}' variable.")
    elif 'tokenizer_qwen14b' in locals():
        print(f"  'tokenizer_{MODEL_KEY_TO_UNLOAD}' variable was already None.")
    else:
        print(f"  'tokenizer_{MODEL_KEY_TO_UNLOAD}' variable not found.")

    # Explicitly set variables to None to prevent accidental reuse
    model_qwen14b = None
    tokenizer_qwen14b = None
    print("  Set model/tokenizer variables to None.")


    # Run Python garbage collector
    print("  Running garbage collection...")
    gc.collect()

    # Clear PyTorch CUDA cache if GPU is available
    if torch.cuda.is_available():
        print("  Clearing PyTorch CUDA cache...")
        torch.cuda.empty_cache()
        print(f"Model {MODEL_KEY_TO_UNLOAD} unloaded and GPU cache cleared.")
        # Optional: Verify memory release
        print("\nGPU Memory Usage after unload:")
        allocated = torch.cuda.memory_allocated(0) / (1024**3)
        reserved = torch.cuda.memory_reserved(0) / (1024**3)
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved: {reserved:.2f} GB")
        # !nvidia-smi
    else:
        print(f"Model {MODEL_KEY_TO_UNLOAD} variables deleted (No GPU cache to clear).")

except NameError as ne:
    warnings.warn(f"NameError during unload: {ne}. Model might have failed loading or was already unloaded.", UserWarning)
except Exception as e:
    warnings.warn(f"An unexpected error occurred during model unloading: {e}", UserWarning)


print("\nModel unloading step complete.")
print("="*60)

In [ ]:
# Cell: Explore and Summarize Qwen-14B Results

from IPython.display import display, Markdown # For better display

print("="*30 + f" Qwen-14B Run: Step 9 - Analyze Individual Results " + "="*30)

# --- Define the key and results list variable for this specific run ---
MODEL_KEY_TO_ANALYZE = MODEL_KEY_QWEN14B #
RESULTS_LIST_VARIABLE = results_qwen14b_json # The list created in the inference loop
DF_RESULTS_VARIABLE_NAME = f"df_results_{MODEL_KEY_TO_ANALYZE}" # e.g., df_results_qwen_14b

print(f"\n--- Exploring Results for {MODEL_KEY_TO_ANALYZE} ---")

# --- Ensure the results list exists ---
if 'RESULTS_LIST_VARIABLE' not in locals() or not RESULTS_LIST_VARIABLE:
    print(f"ERROR: `{RESULTS_LIST_VARIABLE._name_}` list not found or is empty. Please run the inference cell (Cell 6) first.")
else:
    # --- Create DataFrame for this model's results ---
    try:
        # Create a specific DataFrame for this model to avoid conflicts
        df_results_specific = pd.DataFrame(RESULTS_LIST_VARIABLE)
        print(f"Created DataFrame '{DF_RESULTS_VARIABLE_NAME}' from results list. Shape: {df_results_specific.shape}")
        # Assign to the dynamic variable name for use below (optional, but makes code cleaner)
        globals()[DF_RESULTS_VARIABLE_NAME] = df_results_specific

    except Exception as e:
         raise RuntimeError(f"Failed to create DataFrame from results list: {e}")

    # --- Use the specific DataFrame for analysis ---
    if DF_RESULTS_VARIABLE_NAME not in locals():
         raise RuntimeError(f"DataFrame {DF_RESULTS_VARIABLE_NAME} was not created successfully.")

    df_results = locals()[DF_RESULTS_VARIABLE_NAME] # Use the dynamically named DataFrame

    # --- Basic Summary ---
    total_rows = len(df_results)
    print(f"\nTotal Rows Processed by {MODEL_KEY_TO_ANALYZE}: {total_rows}")

    # --- Parsing Success/Failure ---
    # Check if 'parsing_error' column exists
    if 'parsing_error' not in df_results.columns:
         print("WARNING: 'parsing_error' column not found in results DataFrame.")
         num_parsing_errors = 0
         parsing_success = total_rows # Assume success if column missing (less ideal)
    else:
         # Calculate errors (where parsing_error column is NOT null/NaN)
         num_parsing_errors = df_results['parsing_error'].notnull().sum()
         parsing_success = total_rows - num_parsing_errors

    parsing_success_pct = (parsing_success / total_rows * 100) if total_rows > 0 else 0
    parsing_error_pct = (num_parsing_errors / total_rows * 100) if total_rows > 0 else 0
    print(f"Rows with Successful JSON Parse: {parsing_success} ({parsing_success_pct:.1f}%)")
    print(f"Rows with Parsing Errors: {num_parsing_errors} ({parsing_error_pct:.1f}%)")

    if num_parsing_errors > 0 and 'parsing_error' in df_results.columns:
        print("\n--- Sample Rows with Parsing Errors ---")
        display_cols_error = ['query_id', 'product_id', 'query', 'raw_output', 'parsing_error']
        # Ensure columns exist before displaying
        display_cols_error = [col for col in display_cols_error if col in df_results.columns]
        with pd.option_context('display.max_colwidth', 150):
            display(df_results[df_results['parsing_error'].notnull()].head()[display_cols_error])
        print("\nTop 5 Parsing Errors:")
        print(df_results[df_results['parsing_error'].notnull()]['parsing_error'].value_counts().head())
        print("\nReview the raw output for these rows to understand why parsing failed.")

    # --- Accuracy Label Distribution (Only for successfully parsed rows) ---
    # Ensure required columns exist
    if 'parsing_error' in df_results.columns and 'accurate_label' in df_results.columns:
        valid_results_df = df_results[df_results['parsing_error'].isnull() & df_results['accurate_label'].notnull()].copy()
        print(f"\nNumber of rows with valid 'accurate_label' after parsing: {len(valid_results_df)}")

        if not valid_results_df.empty:
            print(f"\n--- {MODEL_KEY_TO_ANALYZE}: Accuracy Label Distribution (Successfully Parsed) ---")
            accuracy_counts = valid_results_df['accurate_label'].value_counts()
            print(accuracy_counts)
            try:
                true_pct = accuracy_counts.get(True, 0) / len(valid_results_df) * 100
                false_pct = accuracy_counts.get(False, 0) / len(valid_results_df) * 100
                print(f"  % Accurate (True): {true_pct:.1f}%")
                print(f"  % Inaccurate (False): {false_pct:.1f}%")
            except ZeroDivisionError:
                print("  Cannot calculate percentages (no valid results).")

            # --- Analysis of Inaccurate Items ---
            df_inaccurate = valid_results_df[valid_results_df['accurate_label'] == False].copy()
            print(f"\n--- {MODEL_KEY_TO_ANALYZE}: Analysis of {len(df_inaccurate)} Items Flagged as Inaccurate ---")
            if not df_inaccurate.empty:
                # Merge context if filtered_df is available
                if 'filtered_df' in locals() and all(c in filtered_df for c in ['query_id', 'product_id', 'llm_product_context']):
                    df_inaccurate = pd.merge(
                        df_inaccurate,
                        filtered_df[['query_id', 'product_id', 'llm_product_context']],
                        on=['query_id', 'product_id'],
                        how='left'
                    )
                    display_cols_inacc = ['query_id', 'product_id', 'query', 'llm_product_context', 'reasoning', 'reformulated_query']
                else:
                    print("  (Skipping merge with llm_product_context as 'filtered_df' is unavailable)")
                    display_cols_inacc = ['query_id', 'product_id', 'query', 'reasoning', 'reformulated_query']

                print("  Displaying Query, Reasoning, and Reformulated Query (Context added if available):")
                with pd.option_context('display.max_colwidth', 200, 'display.max_rows', 50):
                     display(df_inaccurate[[col for col in display_cols_inacc if col in df_inaccurate.columns]]) # Display only existing columns
                print("\n  Review: Is the 'reasoning' valid? Is 'reformulated_query' sensible?")
            else:
                print("  No items flagged as inaccurate.")

            # --- Spot Check: Accurate Items ---
            df_accurate = valid_results_df[valid_results_df['accurate_label'] == True].copy()
            num_accurate_to_show = min(5, len(df_accurate))
            print(f"\n--- {MODEL_KEY_TO_ANALYZE}: Spot Check {num_accurate_to_show} Items Flagged as Accurate ---")
            if not df_accurate.empty:
                 if 'filtered_df' in locals() and all(c in filtered_df for c in ['query_id', 'product_id', 'llm_product_context']):
                     df_accurate_merged = pd.merge(
                        df_accurate.head(num_accurate_to_show),
                        filtered_df[['query_id', 'product_id', 'llm_product_context']],
                        on=['query_id', 'product_id'],
                        how='left'
                    )
                     display_cols_acc = ['query_id', 'product_id', 'query', 'llm_product_context', 'reasoning']
                 else:
                     print("  (Skipping merge with llm_product_context as 'filtered_df' is unavailable)")
                     df_accurate_merged = df_accurate.head(num_accurate_to_show)
                     display_cols_acc = ['query_id', 'product_id', 'query', 'reasoning']

                 print("  Displaying Query and Reasoning (Context added if available):")
                 with pd.option_context('display.max_colwidth', 200):
                     display(df_accurate_merged[[col for col in display_cols_acc if col in df_accurate_merged.columns]])
                 print("\n  Review: Is reasoning correct (spec met vs. spec assumed met)?")
            else:
                 print("  No items flagged as accurate to spot check.")

        else:
            print("\nNo successfully parsed results with valid accuracy labels found to analyze.")
    else:
         print("\nSkipping detailed analysis as 'parsing_error' or 'accurate_label' column is missing.")


    # --- Overall Summary Markdown ---
    valid_results_count = len(valid_results_df) if 'valid_results_df' in locals() else 'N/A'
    inaccurate_count = len(df_inaccurate) if 'df_inaccurate' in locals() and not df_inaccurate.empty else 0
    accurate_count = len(df_accurate) if 'df_accurate' in locals() and not df_accurate.empty else 0 # Get count before head()

    summary_markdown = f"""
### Summary for {MODEL_KEY_TO_ANALYZE} Individual Run:

*   **Total Rows Processed:** {total_rows}
*   **JSON Parsing Success:** {parsing_success}/{total_rows} ({parsing_success_pct:.1%})
*   **Rows with Valid 'accurate_label':** {valid_results_count}
*   **Items Flagged Inaccurate (among valid):** {inaccurate_count}
*   **Items Flagged Accurate (among valid):** {accurate_count}

**Next Steps (After This Analysis):**
1.  Review parsing errors for {MODEL_KEY_TO_ANALYZE}.
2.  Critically evaluate the reasoning for both inaccurate and accurate items for {MODEL_KEY_TO_ANALYZE}.
3.  Proceed to the **Aggregation Steps** (Cells 10-13) to combine these results with other models.
"""
    display(Markdown(summary_markdown))

print(f"\nAnalysis for {MODEL_KEY_TO_ANALYZE} complete.")
print("="*60)

In [ ]:

with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    print(df_results.head(3))

In [ ]:
# Cell : Setup for Gemma-12B Run

print("\n" + "="*30 + " Setting up for Gemma-12B " + "="*30)

# --- Model Configuration ---
MODEL_ID_GEMMA12B = "google/gemma-3-12b-it"
MODEL_KEY_GEMMA12B = "gemma_12b" # Short key for this model

print(f"Next Target Model: {MODEL_KEY_GEMMA12B} ({MODEL_ID_GEMMA12B})")

# --- Quantization (MUST use 4-bit for 12B on 40GB) ---
USE_QUANTIZATION_GEMMA12B = "4bit"
quant_config_gemma12b = None

if USE_QUANTIZATION_GEMMA12B == "4bit":
    quant_config_gemma12b = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    print("Applying 4-bit quantization for Gemma-12B.")
elif USE_QUANTIZATION_GEMMA12B == "8bit":
     quant_config_gemma12b = BitsAndBytesConfig(load_in_8bit=True)
     print("Applying 8-bit quantization for Gemma-12B (May still cause OOM).")
else:
    print("WARNING: No quantization specified for Gemma-12B. This is very likely to cause OutOfMemoryError.")

# --- Ensure necessary functions/variables are defined ---
# Make sure load_llm, SYSTEM_PROMPT, USER_PROMPT_TEMPLATE, run_llm_check_json_chat
# and the filtered_df are available from previous cells. # Colab issues sometimes
if 'load_llm' not in locals(): raise NameError("Function 'load_llm' not defined.")
if 'SYSTEM_PROMPT' not in locals(): raise NameError("Variable 'SYSTEM_PROMPT' not defined.")
if 'USER_PROMPT_TEMPLATE' not in locals(): raise NameError("Variable 'USER_PROMPT_TEMPLATE' not defined.")
if 'run_llm_check_json_chat' not in locals(): raise NameError("Function 'run_llm_check_json_chat' not defined.")
if 'filtered_df' not in locals() or 'llm_product_context' not in filtered_df.columns:
     raise RuntimeError("ERROR: `filtered_df` with `llm_product_context` is not available.")

print("Setup for Gemma-12B complete.")

In [ ]:
# Cell: Load Gemma-12B Model

print(f"\n--- Loading Model: {MODEL_KEY_GEMMA12B} ---")

# Load using the specific config for this model
model_gemma12b, tokenizer_gemma12b = load_llm(MODEL_KEY_GEMMA12B, MODEL_ID_GEMMA12B, quant_config_gemma12b)

# Check if loading was successful
if model_gemma12b is None or tokenizer_gemma12b is None:
    raise RuntimeError(f"Failed to load the model {MODEL_ID_GEMMA12B}. Cannot proceed.")
else:
    print(f"\nModel {MODEL_KEY_GEMMA12B} loaded successfully.")
    # Optional: Print memory footprint
    try:
        print(f"Memory Footprint: {model_gemma12b.get_memory_footprint()} bytes")
    except:
        pass # Ignore if method not available

In [ ]:
# Cell: Run Inference Loop for Gemma-12B

# --- Check if Model is Ready ---
if model_gemma12b is None or tokenizer_gemma12b is None:
     raise RuntimeError(f"ERROR: Model {MODEL_KEY_GEMMA12B} is not loaded properly.")

# --- Run Inference Loop ---
results_gemma12b_json = [] # Store results specifically for this model
print(f"\n--- Starting JSON Inference Loop for {MODEL_KEY_GEMMA12B} ---")

num_rows = len(filtered_df)
for index, row in filtered_df.iterrows():
    print(f"\rProcessing Row {index+1}/{num_rows} with {MODEL_KEY_GEMMA12B}...", end="")

    # Get data for the current row
    query = row['query']
    product_context = row['llm_product_context'] if pd.notna(row['llm_product_context']) and row['llm_product_context'] else "No product information available."

    # Run inference using the JSON chat function
    try:
        # Pass the correct model/tokenizer variables for this run
        output = run_llm_check_json_chat(
            query,
            product_context,
            model_gemma12b, # Use Gemma model
            tokenizer_gemma12b, # Use Gemma tokenizer
            SYSTEM_PROMPT,
            USER_PROMPT_TEMPLATE,
            MODEL_KEY_GEMMA12B # Pass the correct key
        )
        results_gemma12b_json.append({
            # Store results with model key for clarity later if needed
            'query_id': row['query_id'],
            'product_id': row['product_id'],
            'query': query,
            'model_key': MODEL_KEY_GEMMA12B, # Add model key
            'accurate_label': output['is_exact_match'],
            'reasoning': output['reasoning'],
            'reformulated_query': output['reformulated_query'],
            'raw_output': output['raw_output'],
            'parsing_error': output['parsing_error']
        })
        if output['parsing_error']:
             print(f"\nParsing Error on row {index+1} ({MODEL_KEY_GEMMA12B}): {output['parsing_error']}. Raw: '{output['raw_output'][:100]}...'")

    except Exception as e:
         print(f"\nFATAL ERROR during inference for row index {index} ({MODEL_KEY_GEMMA12B}): {e}")
         results_gemma12b_json.append({
            'query_id': row['query_id'],
            'product_id': row['product_id'],
            'query': query,
            'model_key': MODEL_KEY_GEMMA12B,
            'accurate_label': None, 'reasoning': 'Inference Loop Error', 'reformulated_query': None,
            'raw_output': f'Inference Loop Error: {e}', 'parsing_error': f'Outer loop error: {e}'
        })
         # break # Optional: Stop if errors occur

print(f"\n--- JSON Inference Loop Completed for {MODEL_KEY_GEMMA12B} ---")

# Display first few raw results for inspection
print(f"\nSample Raw Results ({MODEL_KEY_GEMMA12B} - first 5 rows):")
print(pd.DataFrame(results_gemma12b_json).head().to_markdown(index=False))

In [ ]:
# Cell: Unload Gemma-12B Model

# Define the key for the model being unloaded (used in print statements)
MODEL_KEY_TO_UNLOAD = "gemma_12b" # Make sure this matches the key used when loading

print(f"\n--- Unloading Model {MODEL_KEY_TO_UNLOAD} ---")
try:
    # Check if the specific variables exist before deleting
    if 'model_gemma12b' in locals() and model_gemma12b is not None:
        del model_gemma12b
        print(f"Deleted 'model_gemma12b' variable.")
    else:
        print(f"'model_gemma12b' variable not found or already None.")

    if 'tokenizer_gemma12b' in locals() and tokenizer_gemma12b is not None:
        del tokenizer_gemma12b
        print(f"Deleted 'tokenizer_gemma12b' variable.")
    else:
         print(f"'tokenizer_gemma12b' variable not found or already None.")

    # Run Python garbage collector
    print("Running garbage collection...")
    gc.collect()

    # Clear PyTorch CUDA cache if GPU is available
    if torch.cuda.is_available():
        print("Clearing PyTorch CUDA cache...")
        torch.cuda.empty_cache()
        print(f"Model {MODEL_KEY_TO_UNLOAD} unloaded and GPU cache cleared.")
        # Optional: Verify memory release with nvidia-smi after running this cell
        # !nvidia-smi
    else:
        print(f"Model {MODEL_KEY_TO_UNLOAD} variables deleted (No GPU cache to clear).")

except NameError as ne:
    warnings.warn(f"NameError during unload: {ne}. Model might have failed loading or was already unloaded.", UserWarning)
except Exception as e:
    warnings.warn(f"An unexpected error occurred during model unloading: {e}", UserWarning)

# Explicitly set variables to None to prevent accidental reuse
model_gemma12b = None
tokenizer_gemma12b = None
print(f"Variables for {MODEL_KEY_TO_UNLOAD} explicitly set to None.")

In [ ]:
# Cell :  Define Final Majority Voting Function

from collections import Counter # M

print("\n" + "="*30 + " Aggregation Step : Defining Majority Voting Logic " + "="*30)

def get_majority_vote_final(row, model_keys):
    """
    Performs majority voting based on results from multiple LLMs for a single item.
    Determines the final accuracy label (True, False, or None for tie/error)
    and the consensus reformulated query if the final label is False.
    Parameters:
        row (pd.Series): A row from the merged DataFrame.
        model_keys (list): A list of model keys participating in the vote.
    Returns:
        pd.Series: Contains final results ('final_accurate', 'consensus_reasoning', etc.).
    """
    votes = []
    reformulations_if_false = []
    reasonings = []
    parsing_errors = []

    for key in model_keys:
        acc_col = f'acc_{key}'
        reason_col = f'reason_{key}'
        reform_col = f'reform_{key}'
        parse_err_col = f'parse_err_{key}'

        # Robustness checks
        if acc_col not in row.index or parse_err_col not in row.index:
             parsing_errors.append(f"{key}: Result columns missing in merged data for this row.")
             continue
        # Use pd.isna() for robust check of missing values (NaN, None)
        if pd.notna(row.get(parse_err_col)): # Check if parsing_error is NOT NaN/None
            parsing_errors.append(f"{key}: Parsing Failed ({row.get(parse_err_col)})")
            continue
        vote_val = row.get(acc_col)
        # Check if vote_val is explicitly boolean True or False
        if not isinstance(vote_val, bool):
            # Use pd.isna() for robust check of missing values (NaN, None) before reporting type
            if pd.isna(vote_val):
                parsing_errors.append(f"{key}: Vote value is missing (NaN/None).")
            else:
                parsing_errors.append(f"{key}: Invalid vote value ({vote_val}, type: {type(vote_val).__name__}). Expected boolean.")
            continue

        # Record valid vote and details
        votes.append(vote_val)
        # Add reasoning, handling potential NaN/None gracefully
        reason_text = row.get(reason_col)
        reasonings.append(f"[{key.upper()} Vote: {vote_val}] {str(reason_text) if pd.notna(reason_text) else 'N/A'}")
        if not vote_val: # If vote is False
            reformulation = row.get(reform_col)
            # Check if reformulation is a non-empty string and not NaN/None
            if pd.notna(reformulation) and isinstance(reformulation, str) and reformulation.strip():
                reformulations_if_false.append(reformulation)

    # Determine final verdict
    num_valid_votes = len(votes)
    final_accurate = None
    final_reformulated = None
    # Combine reasoning regardless of vote outcome; add Tie/Error info later
    consensus_reasoning = "; ".join(reasonings)

    if num_valid_votes == 0:
        # Add specific error info if available
        error_suffix = f" Errors: {'; '.join(parsing_errors)}" if parsing_errors else ""
        consensus_reasoning = "No valid votes received from any model." + error_suffix
        # final_accurate remains None
    else:
        num_true = sum(votes)
        num_false = num_valid_votes - num_true
        if num_true > num_false:
            final_accurate = True
        elif num_false > num_true:
            final_accurate = False
        else: # Tie
            # Append tie info to reasoning string
            consensus_reasoning += "; [Result: Tied Vote]"
            # final_accurate remains None (indicating tie)

        # Determine consensus reformulation only if final vote is False
        if final_accurate is False and reformulations_if_false:
            reform_counts = Counter(reformulations_if_false)
            if reform_counts:
                # Get the most frequent reformulation among False voters
                final_reformulated = reform_counts.most_common(1)[0][0]

    # Compile voting/parsing errors into a single string (or None)
    voting_errors_str = "; ".join(parsing_errors) if parsing_errors else None

    return pd.Series([final_accurate, consensus_reasoning, final_reformulated, voting_errors_str],
                     index=['final_accurate', 'consensus_reasoning', 'final_reformulated', 'voting_errors'])

print("Majority voting function 'get_majority_vote_final' defined.")
print("="*60)

In [ ]:
# Cell : Combine Results and Perform Majority Voting (Qwen-14B + Gemma-12B) first twp models


print("\n" + "="*30 + " Combining and Aggregating Results (Qwen-14B & Gemma-12B) " + "="*30)

# --- Define the models and their results variables for THIS specific aggregation ---
MODELS_TO_AGGREGATE = {
    'qwen_14b': 'results_qwen14b_json',  #
    'gemma12b': 'results_gemma12b_json'   #
}

# --- Gather available results lists ---
print("Gathering results for models to aggregate:")
final_model_results_lists = {}
FINAL_MODEL_KEYS = [] # List of keys for models successfully found

for model_key, results_var_name in MODELS_TO_AGGREGATE.items():
    print(f"  Checking for: {model_key} (Variable: {results_var_name})")
    # Check if the variable exists in the local scope, is a list, and is not empty
    if results_var_name in locals() and isinstance(locals()[results_var_name], list) and locals()[results_var_name]:
        results_list = locals()[results_var_name]
        # Further validation: check if it's a list of dictionaries
        if results_list and isinstance(results_list[0], dict):
            final_model_results_lists[model_key] = results_list
            FINAL_MODEL_KEYS.append(model_key)
            print(f"    [OK] Found {len(results_list)} results.")
        else:
             print(f"    [WARNING] Variable '{results_var_name}' found but is not a valid list of dictionaries. Excluding {model_key}.")
    else:
        print(f"    [WARNING] Results variable '{results_var_name}' not found or empty. Excluding {model_key}.")


# --- Check if enough models were found ---
# This logic remains the same, it checks the length of the dynamically built list
if len(FINAL_MODEL_KEYS) < 2:
     raise SystemExit(f"ERROR: Found valid results for only {len(FINAL_MODEL_KEYS)} model(s) ({FINAL_MODEL_KEYS}). Need results from at least 2 models to perform majority voting. Aggregation skipped.")
else:
    print(f"\nProceeding with aggregation using {len(FINAL_MODEL_KEYS)} models: {FINAL_MODEL_KEYS}")


# --- Merge Results from Found Models ---
# This section uses the dynamically determined FINAL_MODEL_KEYS list
print("\n--- Merging results from available models ---")
df_merged_all_final = None # Initialize the final merged DataFrame

for i, model_key in enumerate(FINAL_MODEL_KEYS):
    # This loop now correctly iterates through ['qwen_14b', 'gemma12b'] if both were found
    results_list = final_model_results_lists[model_key]
    print(f"  Processing results for merging: {model_key}")
    try:
        df_model = pd.DataFrame(results_list)

        # --- Validate essential columns ---
        required_cols = ['query_id', 'product_id', 'accurate_label', 'reformulated_query', 'reasoning', 'parsing_error']
        if not all(col in df_model.columns for col in required_cols):
             missing = [col for col in required_cols if col not in df_model.columns]
             print(f"    [ERROR] Missing required columns for {model_key}: {missing}. Skipping merge for this model.")
             FINAL_MODEL_KEYS.remove(model_key) # Remove key if columns are missing
             continue

        # --- Select and Rename (Dynamically uses model_key)---
        df_model_renamed = df_model[required_cols].rename(columns={
            'accurate_label': f'acc_{model_key}',
            'reformulated_query': f'reform_{model_key}',
            'reasoning': f'reason_{model_key}',
            'parsing_error': f'parse_err_{model_key}'
        })
        # print(f"    Renamed columns for {model_key}.") # Optional debug print

        # --- Merge ---
        if df_merged_all_final is None:
            df_merged_all_final = df_model_renamed
            print(f"    Initialized merged DataFrame.")
        else:
            with warnings.catch_warnings():
                warnings.simplefilter(action='ignore', category=FutureWarning)
                df_merged_all_final = pd.merge(
                    df_merged_all_final,
                    df_model_renamed,
                    on=['query_id', 'product_id'],
                    how='outer'
                )
            print(f"    Merged with previous results.")

    except Exception as e:
        print(f"    [ERROR] Failed to process or merge results for model {model_key}: {e}")
        if model_key in FINAL_MODEL_KEYS: FINAL_MODEL_KEYS.remove(model_key)


# --- Final Check After Merging ---
if df_merged_all_final is None:
    raise SystemExit("FATAL ERROR: Failed to create a merged DataFrame.")
elif len(FINAL_MODEL_KEYS) < 2:
     raise SystemExit(f"ERROR: After merge attempts, only {len(FINAL_MODEL_KEYS)} valid model(s) remain ({FINAL_MODEL_KEYS}). Need at least 2 for voting.")
else:
    print(f"\nSuccessfully merged results for {len(FINAL_MODEL_KEYS)} models: {FINAL_MODEL_KEYS}")
    print(f"  Merged DataFrame shape: {df_merged_all_final.shape}")
    print("--- Preview of Merged DataFrame (First 5 Rows) ---")
    cols_to_preview = ['query_id', 'product_id'] + [f'acc_{k}' for k in FINAL_MODEL_KEYS]
    display(df_merged_all_final[[col for col in cols_to_preview if col in df_merged_all_final.columns]].head())


# --- Majority Voting Logic (Define the function - same as before) ---
# Ensure the function 'get_majority_vote_final' from Cell 12 is defined
if 'get_majority_vote_final' not in locals() or not callable(get_majority_vote_final):
     # If not defined, copy the function definition from Cell 12 here or ensure Cell 12 was run
     raise NameError("Voting function 'get_majority_vote_final' is not defined. Please run Cell 12 first.")
else:
     print("\nUsing previously defined 'get_majority_vote_final' function for voting.")


# --- Apply Voting ---
# This automatically uses the corrected FINAL_MODEL_KEYS list
print(f"\nApplying majority vote using models: {FINAL_MODEL_KEYS}...")
vote_results = df_merged_all_final.apply(
    lambda row: get_majority_vote_final(row, FINAL_MODEL_KEYS), axis=1
)
print("Voting function applied.")


# --- Combine votes with identifiers and original query ---
df_final_voted = pd.concat([df_merged_all_final[['query_id', 'product_id']], vote_results], axis=1)

if 'filtered_df' in locals() and all(c in filtered_df for c in ['query_id', 'product_id', 'query']):
    print("Merging back original query text...")
    df_final_voted = pd.merge(
        df_final_voted,
        filtered_df[['query_id', 'product_id', 'query']].drop_duplicates(),
        on=['query_id', 'product_id'],
        how='left'
    )
else:
    print("[INFO] Original query text not merged back.")


# --- Prepare Final Output Table ---
print("\n--- Preparing Final Output Table ---")
df_output_voted = df_final_voted.rename(columns={
    'final_accurate': 'accurate_label',
    'final_reformulated': 'reformulated_query'
})

final_deliverable_cols = ['query_id', 'product_id', 'accurate_label', 'reformulated_query']
context_cols = ['query', 'consensus_reasoning', 'voting_errors']
display_cols_final = final_deliverable_cols + [col for col in context_cols if col in df_output_voted.columns]
df_output_final_display = df_output_voted[display_cols_final]


print("\n--- Final Results After Majority Voting (Qwen-14B & Gemma-12B) ---")
with pd.option_context('display.max_rows', 25, 'display.max_columns', None, 'display.width', 1200):
    display(df_output_final_display)


# --- Save Final Voted Results ---
# UPDATE Filename to reflect the models used
output_filename_voted = f"grainger_llm_verification_results_{'_'.join(FINAL_MODEL_KEYS)}_majority_vote.csv"
print(f"\nAttempting to save final deliverable results to: {output_filename_voted}")
try:
    if all(col in df_output_voted.columns for col in final_deliverable_cols):
        df_to_save = df_output_voted[final_deliverable_cols]
        df_to_save.to_csv(output_filename_voted, index=False)
        print(f"  Successfully saved {output_filename_voted}")
    else:
        missing_save_cols = [c for c in final_deliverable_cols if c not in df_output_voted.columns]
        print(f"  [ERROR] Cannot save final deliverable CSV. Missing columns: {missing_save_cols}")
except Exception as e:
    print(f"  [ERROR] Error saving final results to CSV: {e}")


# --- Optional: Analysis of Voted Inaccurate Items ---
# This uses the df_output_voted created above, which is based on the correct models
print("\n--- Analysis of Inaccurate Items (Based on Majority Vote) ---")
if 'accurate_label' in df_output_voted.columns:
    df_voted_inaccurate = df_output_voted[df_output_voted['accurate_label'] == False].copy()
    if not df_voted_inaccurate.empty:
         print(f"Found {len(df_voted_inaccurate)} items flagged as inaccurate by majority vote:")
         # Merge back individual model details from df_merged_all_final
         df_voted_inaccurate_details = pd.merge(
             df_voted_inaccurate[['query_id', 'product_id', 'query', 'reformulated_query', 'consensus_reasoning']],
             df_merged_all_final, # Contains individual acc_*, reason_*, etc.
             on=['query_id', 'product_id'],
             how='left'
         )
         with pd.option_context('display.max_rows', 50, 'display.max_columns', None, 'display.width', 1500):
            display(df_voted_inaccurate_details)
    else:
         print("No items flagged as inaccurate by majority vote.")
else:
     print("Skipping analysis of inaccurate items ('accurate_label' column not found).")


print("\nCombination, Voting, and Saving step complete.")
print("="*60)

In [ ]:
# Cell: Aggregated Results Exploration (Corrected & Complete)

# --- Configuration ---
# created in our previous cell where I merged and applied voting.
MERGED_DF_NAME = 'df_merged_all_final' # DataFrame with individual model results (acc_model, reason_model, etc.)
VOTED_DF_NAME = 'df_output_voted'       # DataFrame with the final voted results (final_accurate, etc.)

# Check if the necessary DataFrames exist
if VOTED_DF_NAME not in locals() or MERGED_DF_NAME not in locals():
    raise NameError(f"ERROR: Required DataFrames ('{VOTED_DF_NAME}', '{MERGED_DF_NAME}') not found. Please ensure the previous Combine/Vote cell ran successfully and created these DataFrames.")

# Check for original filtered data (optional but good for context)
if 'filtered_df' not in locals():
     warnings.warn("WARNING: Original 'filtered_df' not found. Query text cannot be merged into detailed views.", UserWarning)
     # Define as None to handle downstream checks gracefully if missing
     original_filtered_df = None
else:
     original_filtered_df = filtered_df # Use the existing filtered_df

# --- Use the correct DataFrame variables ---
# Access the DataFrames using locals() to use the defined names
df_output_voted = locals()[VOTED_DF_NAME]
df_merged_all = locals()[MERGED_DF_NAME]

print("\n" + "="*30 + " Aggregation Step: Explore Aggregated Results " + "="*30)

# --- 1. Overall Summary ---
total_rows = len(df_output_voted)

# Dynamically get model keys from the columns of the merged df
model_keys_used = []
if df_merged_all is not None:
     # Extract keys assuming 'acc_MODELKEY' format
     potential_keys = [col.split('acc_')[1] for col in df_merged_all.columns if col.startswith('acc_')]
     # Get unique keys and sort them for consistent order
     model_keys_used = sorted(list(set(potential_keys)))

# Fallback if keys couldn't be determined
if not model_keys_used:
     # Use FINAL_MODEL_KEYS from config if available, otherwise use a placeholder
     model_keys_used = config.LLM_CONFIG.keys() if 'config' in locals() and hasattr(config, 'LLM_CONFIG') else ["Unknown Models"]
     print("WARNING: Could not determine model keys from merged DataFrame columns. Using fallback keys.")

print(f"Analysis based on models: {model_keys_used}")
print(f"Total Rows Analyzed: {total_rows}")

# Check for voting errors reported in the final voted DataFrame
if 'voting_errors' in df_output_voted.columns:
    # Count rows where voting_errors is not null/NaN
    voting_errors_count = df_output_voted['voting_errors'].notna().sum()
    # Ensure it's an integer
    voting_errors_count = int(voting_errors_count) if not isinstance(voting_errors_count, int) else voting_errors_count
    print(f"Rows with Voting/Parsing Errors reported during vote: {voting_errors_count}")
else:
    voting_errors_count = 'N/A (column missing)'
    print("Rows with Voting/Parsing Errors reported during vote: N/A ('voting_errors' column missing)")


# Calculate final label distribution from the voted DataFrame
final_counts = pd.Series(dtype=int) # Initialize empty series
if 'accurate_label' in df_output_voted.columns:
    # Count True, False, and NaN (nulls) separately
    final_counts = df_output_voted['accurate_label'].value_counts(dropna=False).rename(
        {True: 'Accurate', False: 'Inaccurate', np.nan: 'Tied/None/Error'} # Rename index labels for clarity
    )
    print("\n--- Final Label Distribution (Majority Vote) ---")
    print(final_counts)

    # Display percentages
    print("\n--- Percentages ---")
    if total_rows > 0:
        for label, count in final_counts.items():
            percentage = (count / total_rows) * 100
            print(f"  % {label}: {percentage:.1f}%")
    else:
        print("  Cannot calculate percentages (0 rows).")
else:
    print("\n--- Final Label Distribution: N/A ('accurate_label' column missing in voted results) ---")


# --- 2. Disagreements / Undecided Cases Analysis ---
print("\n" + "="*30 + " Focus: Undecided Cases (Tied Votes or Errors) " + "="*30)
num_undecided = 0 # Default
if 'accurate_label' in df_output_voted.columns:
    # Filter for rows where the final accurate_label is null
    df_undecided = df_output_voted[df_output_voted['accurate_label'].isnull()].copy()
    num_undecided = len(df_undecided)
    print(f"Number of items where final vote was Tied or resulted in Error (label is None): {num_undecided}")

    if num_undecided > 0:
        # Merge back individual model results for detailed comparison
        df_undecided_details = pd.merge(
            df_undecided[['query_id', 'product_id']], # Start with identifiers of undecided rows
            df_merged_all,                             # Merge with the DF containing individual results
            on=['query_id', 'product_id'],
            how='left'
        )
        # Merge back context columns from the final voted df (consensus reason, errors, query)
        merge_cols_context = ['query_id', 'product_id', 'consensus_reasoning', 'voting_errors']
        # Add original query if available
        if original_filtered_df is not None and 'query' in original_filtered_df.columns:
            df_undecided_details = pd.merge(
                 df_undecided_details,
                 original_filtered_df[['query_id', 'product_id', 'query']].drop_duplicates(),
                 on=['query_id', 'product_id'],
                 how='left'
            )
            if 'query' not in merge_cols_context: merge_cols_context.append('query')


        df_undecided_details = pd.merge(
            df_undecided_details,
            df_output_voted[[col for col in merge_cols_context if col in df_output_voted.columns]], # Select existing context cols
            on=['query_id', 'product_id'],
            how='left'
        )


        # Define columns to display for undecided cases
        display_cols_undecided = ['query_id', 'product_id']
        if 'query' in df_undecided_details.columns: display_cols_undecided.append('query')
        # Add columns for each model used
        for key in model_keys_used:
            display_cols_undecided.extend([f'acc_{key}', f'reason_{key}', f'parse_err_{key}'])
        # Add final consensus and error columns
        display_cols_undecided.extend(['consensus_reasoning', 'voting_errors'])

        # Filter list to only include columns that actually exist in the DataFrame
        existing_display_cols_und = [col for col in display_cols_undecided if col in df_undecided_details.columns]

        print("\n--- Details of Undecided Cases ---")
        print("Review individual model votes, reasoning, and any reported errors:")
        with pd.option_context('display.max_colwidth', 180, 'display.max_rows', 50, 'display.width', 1200):
            display(df_undecided_details[existing_display_cols_und])
    # else: # Already printed "Number of items..." is 0
    #     print("No undecided cases found.")
else:
    print("Skipping analysis of undecided cases ('accurate_label' column missing).")
    num_undecided = 'N/A' # Set for summary


# --- 3. Consensus Inaccurate Analysis ---
print("\n" + "="*30 + " Focus: Consensus Inaccurate " + "="*30)
num_consensus_inaccurate = 0 # Default
if 'accurate_label' in df_output_voted.columns:
    # Filter for rows where the final label is False
    df_consensus_inaccurate = df_output_voted[df_output_voted['accurate_label'] == False].copy()
    num_consensus_inaccurate = len(df_consensus_inaccurate)
    print(f"Number of items where final vote was 'Inaccurate': {num_consensus_inaccurate}")

    if num_consensus_inaccurate > 0:
        print("\n--- Details of Consensus Inaccurate ---")
        # Define columns to display
        display_cols_inaccurate = ['query_id', 'product_id', 'query', 'reformulated_query', 'consensus_reasoning']
        # Filter for columns that actually exist in the DataFrame
        existing_display_cols_inacc = [col for col in display_cols_inaccurate if col in df_consensus_inaccurate.columns]
        with pd.option_context('display.max_colwidth', 180, 'display.max_rows', 50, 'display.width', 1200):
            display(df_consensus_inaccurate[existing_display_cols_inacc])
        print("\nReview the consensus reasoning and the proposed reformulated query.")
    # else: # Already printed "Number of items..." is 0
    #     print("No items found with consensus 'Inaccurate'.")
else:
     print("Skipping analysis of inaccurate items ('accurate_label' column missing).")
     num_consensus_inaccurate = 'N/A'


# --- 4. Consensus Accurate Analysis ---
print("\n" + "="*30 + " Focus: Consensus Accurate " + "="*30)
num_consensus_accurate = 0 # Default
if 'accurate_label' in df_output_voted.columns:
    # Filter for rows where the final label is True
    df_consensus_accurate = df_output_voted[df_output_voted['accurate_label'] == True].copy()
    num_consensus_accurate = len(df_consensus_accurate)
    print(f"Number of items where final vote was 'Accurate': {num_consensus_accurate}")

    if num_consensus_accurate > 0:
        # Define columns to display
        display_cols_accurate = ['query_id', 'product_id', 'query', 'consensus_reasoning']
        # Filter for columns that actually exist
        existing_display_cols_acc = [col for col in display_cols_accurate if col in df_consensus_accurate.columns]
        # Show a sample
        num_to_show = min(10, num_consensus_accurate)
        print(f"\n--- Details of Consensus Accurate (Showing up to {num_to_show}) ---")
        with pd.option_context('display.max_colwidth', 180, 'display.width', 1200):
            display(df_consensus_accurate.head(num_to_show)[existing_display_cols_acc])
        print("\nReview the consensus reasoning (should confirm rules were met or assumed met).")
    # else: # Already printed "Number of items..." is 0
    #     print("No items found with consensus 'Accurate'.")
else:
    print("Skipping analysis of accurate items ('accurate_label' column missing).")
    num_consensus_accurate = 'N/A'


# --- 5. Final Summary Markdown (Corrected Percentage Formatting) ---
print("\n" + "="*30 + " Overall Summary Markdown (Aggregated) " + "="*30)

# Retrieve counts safely using .get() with the RENAMED labels from final_counts
accurate_count = final_counts.get('Accurate', 0) if isinstance(final_counts, pd.Series) else 'N/A'
inaccurate_count = final_counts.get('Inaccurate', 0) if isinstance(final_counts, pd.Series) else 'N/A'
undecided_error_count = final_counts.get('Tied/None/Error', 0) if isinstance(final_counts, pd.Series) else 'N/A'

# Determine filename for display (ensure variable exists from saving step)
csv_filename_display = config.FINAL_OUTPUT_CSV if 'config' in locals() and hasattr(config, 'FINAL_OUTPUT_CSV') else 'FILENAME_NOT_SET'
if 'output_filename_final' in locals(): # Check if explicitly set in a previous cell
     csv_filename_display = output_filename_final

# Helper function to calculate and format percentage string safely
def format_percentage_string(count, total):
    """Calculates percentage and formats to string 'XX.X%', or returns 'N/A'."""
    if isinstance(count, int) and isinstance(total, int) and total > 0:
        percentage_value = (count / total) # Calculate ratio
        return f"{percentage_value:.1%}" # Format as percentage string
    else:
        return "N/A" # Return string 'N/A' if calculation not possible

# Construct the Markdown string using the helper function for percentages
summary_md = f"""
### Final Summary: LLM Verification Aggregated Results (Models: {', '.join(model_keys_used)})

*   **Total Items Processed:** {total_rows}
*   **Final Vote Distribution:**
    *   Accurate (True): {accurate_count} ({format_percentage_string(accurate_count, total_rows)})
    *   Inaccurate (False): {inaccurate_count} ({format_percentage_string(inaccurate_count, total_rows)})
    *   Tied / Undecided / Error (None): {undecided_error_count} ({format_percentage_string(undecided_error_count, total_rows)})
*   **Items with Voting/Parsing Errors Reported during vote:** {voting_errors_count}

**Key Areas for Review:**

1.  **Undecided Cases ({num_undecided} items):** Rows where the final label is `None`, likely due to tied votes (if using an even number of models) or errors preventing a majority. Manual review of individual model results is needed. See 'Focus: Undecided Cases' table.
2.  **Consensus Inaccurate ({num_consensus_inaccurate} items):** Items where the final vote was 'False'. Review the consensus reasoning and the proposed `reformulated_query`. See 'Focus: Consensus Inaccurate' table.
3.  **Consensus Accurate ({num_consensus_accurate} items):** Items where the final vote was 'True'. Spot-check the consensus reasoning to ensure rules were applied correctly. See 'Focus: Consensus Accurate' table.

**CSV Output:** The file `{csv_filename_display}` contains the final results (4 required columns: `query_id`, `product_id`, `accurate_label`, `reformulated_query`). Rows with undecided votes will have a blank/null value in the `accurate_label` column.
"""
display(Markdown(summary_md))
print("="*60)


In [ ]:

print("\n" + "="*30 + " Displaying Full Disagreement Details " + "="*30)

# --- Check if the disagreement details DataFrame exists ---
if 'df_undecided_details' not in locals():
    print("ERROR: DataFrame 'df_undecided_details' not found.")
    print("Please run the 'Inspect Disagreements' cell first to generate it.")
elif df_undecided_details.empty:
    print("No disagreements were found, so there are no details to display.")
else:
    print(f"Displaying full details for the {len(df_undecided_details)} disagreement row(s):")
    print("(Scroll horizontally to see all model details and reasoning)")

    # --- Use pd.option_context to display full content ---
    with pd.option_context(
        'display.max_columns', None,    # Show all columns
        'display.max_colwidth', None,   # Show full content of each column
        'display.width', 2000           # Set a large width for the display area
        ):
        display(df_undecided_details) # Use display() for better notebook rendering

print("\n" + "="*60)

In [ ]:
# Cell: Update load_llm Function Definition

# Keep the simple cache dictionary if you used it before
# loaded_models = {} # Re-initialize if needed, or remove if not using caching

def load_llm(model_id_key, model_id_str, quantization_cfg):
    """
    Loads a specified LLM and tokenizer, applying quantization if configured.
    Handles potential memory issues and model loading errors.
    Updated to handle trust_remote_code for common model types.

    Parameters:
        model_id_key (str): A short key for the model (e.g., 'qwen_7b', 'mistral_small').
        model_id_str (str): The Hugging Face model identifier.
        quantization_cfg (BitsAndBytesConfig | None): The quantization config object.

    Returns:
        tuple: (model, tokenizer) or (None, None) if loading fails.
    """
    print(f"\nAttempting to load model: {model_id_str}...")
    model = None
    tokenizer = None
    try:
        # Determine if trust_remote_code is needed (common for Qwen, Mistral, etc.)
        # Convert to lowercase for robust checking
        model_id_lower = model_id_str.lower()
        trust_remote = any(trust_name in model_id_lower for trust_name in ["qwen", "mistral"])
        if trust_remote:
            print(f"Setting trust_remote_code=True for model: {model_id_str}")
        else:
             print(f"Setting trust_remote_code=False for model: {model_id_str}")

        tokenizer = AutoTokenizer.from_pretrained(model_id_str, trust_remote_code=trust_remote)

        # Determine torch_dtype
        compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
        print(f"Using compute dtype: {compute_dtype}")

        model = AutoModelForCausalLM.from_pretrained(
            model_id_str,
            device_map="auto",
            torch_dtype=compute_dtype,
            quantization_config=quantization_cfg,
            trust_remote_code=trust_remote,
        )

        print(f"Successfully loaded {model_id_str}.")
        return model, tokenizer

    except torch.cuda.OutOfMemoryError:
        error_msg = f"ERROR: CUDA Out of Memory loading {model_id_str}."
        if quantization_cfg is None:
             error_msg += " Consider enabling quantization (e.g., 4bit)."
        else:
             error_msg += " Even with quantization, memory is insufficient. Try a smaller model or different hardware."
        print(error_msg)
        # Clean up memory aggressively
        del model
        del tokenizer
        gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        return None, None
    except Exception as e:
        print(f"ERROR: Failed to load model {model_id_str}. Error: {e}")
        # Clean up memory aggressively
        del model
        del tokenizer
        gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        return None, None

print("Updated 'load_llm' function defined.")

In [ ]:
# Cell: Setup for Mistral-Small-3.1 Run

print("\n" + "="*30 + " Setting up for Mistral-Small-3.1 " + "="*30)

# --- Model Configuration ---
MODEL_ID_MISTRAL_SMALL = "mistralai/Mistral-Small-24B-Instruct-2501"
MODEL_KEY_MISTRAL_SMALL = "mistral_small" # Short key

print(f"Target Model: {MODEL_KEY_MISTRAL_SMALL} ({MODEL_ID_MISTRAL_SMALL})")

# --- Quantization (MUST use 4-bit for 24B on 40GB A100) ---
USE_QUANTIZATION_MISTRAL_SMALL = "4bit" # Enforce 4-bit
quant_config_mistral_small = None

if USE_QUANTIZATION_MISTRAL_SMALL == "4bit":
    # Ensure BitsAndBytesConfig is imported
    from transformers import BitsAndBytesConfig
    quant_config_mistral_small = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    print("Applying 4-bit quantization for Mistral-Small-3.1.")
else:
    # This branch should ideally not be hit given the hardware constraints
    print("WARNING: Not using 4-bit quantization for 24B model. High risk of OOM.")

# --- Ensure necessary functions/variables are defined ---
# Basic checks to catch issues early
required_vars = ['load_llm', 'SYSTEM_PROMPT', 'USER_PROMPT_TEMPLATE',
                 'run_llm_check_json_chat', 'filtered_df']
for var in required_vars:
    if var not in locals():
        raise NameError(f"Required function or variable '{var}' not defined. Please run previous cells.")
if 'llm_product_context' not in filtered_df.columns:
     raise RuntimeError("ERROR: `filtered_df` is missing `llm_product_context` column.")

print("Setup for Mistral-Small-3.1 complete.")

In [ ]:
# Cell: Load Mistral-Small-3.1 Model

print(f"\n--- Loading Model: {MODEL_KEY_MISTRAL_SMALL} ---")

# Explicitly clear cache before loading a large model
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Cleared CUDA cache before loading.")

# Load using the updated load_llm function and specific config
model_mistral_small, tokenizer_mistral_small = load_llm(
    MODEL_KEY_MISTRAL_SMALL,
    MODEL_ID_MISTRAL_SMALL,
    quant_config_mistral_small # Pass the 4-bit config
)

# Check if loading was successful
if model_mistral_small is None or tokenizer_mistral_small is None:
    raise RuntimeError(f"Failed to load the model {MODEL_ID_MISTRAL_SMALL}. Cannot proceed. Check logs for OOM or other errors.")
else:
    print(f"\nModel {MODEL_KEY_MISTRAL_SMALL} loaded successfully.")
    # Optional: Print memory footprint (if method exists)
    try:
        print(f"Memory Footprint: {model_mistral_small.get_memory_footprint()} bytes")
    except AttributeError:
        print("Model does not have get_memory_footprint method.")
    except Exception as e:
        print(f"Could not get memory footprint: {e}")

    # Optional: Display GPU memory usage after loading
    if torch.cuda.is_available():
        print("\nGPU Memory Usage after load:")
        # Simple check using torch functions
        allocated = torch.cuda.memory_allocated(0) / (1024**3)
        reserved = torch.cuda.memory_reserved(0) / (1024**3)
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved: {reserved:.2f} GB")
        # Or use nvidia-smi command if preferred
        # !nvidia-smi

In [ ]:
# Cell: Run Inference Loop for Mistral-Small-3.1

# --- Check if Model is Ready ---
if 'model_mistral_small' not in locals() or model_mistral_small is None or \
   'tokenizer_mistral_small' not in locals() or tokenizer_mistral_small is None:
     raise RuntimeError(f"ERROR: Model {MODEL_KEY_MISTRAL_SMALL} is not loaded properly.")

# --- Run Inference Loop ---
results_mistral_small_json = [] # Store results specifically for this model
print(f"\n--- Starting JSON Inference Loop for {MODEL_KEY_MISTRAL_SMALL} ---")

num_rows = len(filtered_df)
from tqdm.auto import tqdm # Add progress bar

start_time = time.time()

# Use tqdm for a progress bar
for index, row in tqdm(filtered_df.iterrows(), total=num_rows, desc=f"Processing {MODEL_KEY_MISTRAL_SMALL}"):
    # print(f"\rProcessing Row {index+1}/{num_rows} with {MODEL_KEY_MISTRAL_SMALL}...", end="") # tqdm replaces this

    # Get data for the current row
    query = row['query']
    product_context = row['llm_product_context'] if pd.notna(row['llm_product_context']) and row['llm_product_context'] else "No product information available."

    # Run inference using the JSON chat function
    try:
        # Pass the correct model/tokenizer variables for this run
        output = run_llm_check_json_chat(
            query,
            product_context,
            model_mistral_small,    # Use Mistral model
            tokenizer_mistral_small,# Use Mistral tokenizer
            SYSTEM_PROMPT,
            USER_PROMPT_TEMPLATE,
            MODEL_KEY_MISTRAL_SMALL # Pass the correct key
        )
        results_mistral_small_json.append({
            # Store results with model key for clarity later if needed
            'query_id': row['query_id'],
            'product_id': row['product_id'],
            'query': query,
            'model_key': MODEL_KEY_MISTRAL_SMALL, # Add model key
            'accurate_label': output['is_exact_match'],
            'reasoning': output['reasoning'],
            'reformulated_query': output['reformulated_query'],
            'raw_output': output['raw_output'],
            'parsing_error': output['parsing_error']
        })
        # Log parsing errors immediately if they occur
        if output['parsing_error']:
             tqdm.write(f"Parsing Error on row {index+1} ({MODEL_KEY_MISTRAL_SMALL}): {output['parsing_error']}. Raw: '{output['raw_output'][:100]}...'")

    except Exception as e:
         # Log fatal errors immediately
         tqdm.write(f"FATAL ERROR during inference for row index {index} ({MODEL_KEY_MISTRAL_SMALL}): {e}")
         results_mistral_small_json.append({
            'query_id': row['query_id'],
            'product_id': row['product_id'],
            'query': query,
            'model_key': MODEL_KEY_MISTRAL_SMALL,
            'accurate_label': None, 'reasoning': 'Inference Loop Error', 'reformulated_query': None,
            'raw_output': f'Inference Loop Error: {e}', 'parsing_error': f'Outer loop error: {e}'
        })
         # break # Optional: Stop if errors occur repeatedly

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\n--- JSON Inference Loop Completed for {MODEL_KEY_MISTRAL_SMALL} ---")
print(f"Total time taken: {elapsed_time:.2f} seconds ({elapsed_time/num_rows:.2f} seconds/row avg)")

# Display first few raw results for inspection
print(f"\nSample Raw Results ({MODEL_KEY_MISTRAL_SMALL} - first 5 rows):")
if results_mistral_small_json:
    print(pd.DataFrame(results_mistral_small_json).head().to_markdown(index=False))
else:
    print("No results generated.")

In [ ]:
# Cell: Unload Mistral-Small-3.1 Model

MODEL_KEY_TO_UNLOAD = "mistral_small" #  key used

print(f"\n--- Unloading Model {MODEL_KEY_TO_UNLOAD} ---")
try:
    # Check if the specific variables exist before deleting
    if 'model_mistral_small' in locals() and model_mistral_small is not None:
        del model_mistral_small
        print(f"Deleted 'model_{MODEL_KEY_TO_UNLOAD}' variable.")
    else:
        print(f"'model_{MODEL_KEY_TO_UNLOAD}' variable not found or already None.")

    if 'tokenizer_mistral_small' in locals() and tokenizer_mistral_small is not None:
        del tokenizer_mistral_small
        print(f"Deleted 'tokenizer_{MODEL_KEY_TO_UNLOAD}' variable.")
    else:
         print(f"'tokenizer_{MODEL_KEY_TO_UNLOAD}' variable not found or already None.")

    # Run Python garbage collector
    print("Running garbage collection...")
    gc.collect()

    # Clear PyTorch CUDA cache if GPU is available
    if torch.cuda.is_available():
        print("Clearing PyTorch CUDA cache...")
        torch.cuda.empty_cache()
        print(f"Model {MODEL_KEY_TO_UNLOAD} unloaded and GPU cache cleared.")
        # Optional: Verify memory release
        print("\nGPU Memory Usage after unload:")
        allocated = torch.cuda.memory_allocated(0) / (1024**3)
        reserved = torch.cuda.memory_reserved(0) / (1024**3)
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved: {reserved:.2f} GB")
        # !nvidia-smi
    else:
        print(f"Model {MODEL_KEY_TO_UNLOAD} variables deleted (No GPU cache to clear).")

except NameError as ne:
    warnings.warn(f"NameError during unload: {ne}. Model might have failed loading or was already unloaded.", UserWarning)
except Exception as e:
    warnings.warn(f"An unexpected error occurred during model unloading: {e}", UserWarning)

# Explicitly set variables to None to prevent accidental reuse
model_mistral_small = None
tokenizer_mistral_small = None
print(f"Variables for {MODEL_KEY_TO_UNLOAD} explicitly set to None.")

In [ ]:
# Cell: Gather and Verify Final Model Results (Corrected for Qwen-14B, Gemma-12B, Mistral-Small)

print("\n" + "="*30 + " Step 1: Gathering Final Model Results " + "="*30)

# --- Define the EXACT model keys used in the final run ---
# These should match the keys used in config and the results list variable names.
FINAL_MODEL_KEYS = ['qwen_14b', 'gemma_12b', 'mistral_small'] # <<< KEYS

# --- Create a dictionary to hold the results lists ---
final_model_results_lists = {}

# --- List to keep track of keys actually found ---
# This will be used later for merging and voting
# to avoid errors if a model's results are missing.
keys_found = []

# --- Load or verify results for each final model ---
print("Checking availability of results for each model...")

# Qwen-14B
# Check if 'results_qwen14b_json' variable exists and is a non-empty list
if 'results_qwen14b_json' in locals() and isinstance(results_qwen14b_json, list) and results_qwen14b_json:
    final_model_results_lists['qwen_14b'] = results_qwen14b_json
    keys_found.append('qwen_14b')
    print(f"  [OK] Found {len(results_qwen14b_json)} results for qwen_14b.")
else:
    print(f"  [WARNING] Results for qwen_14b ('results_qwen14b_json') not found or empty. It will be excluded.")
    # No need to remove from FINAL_MODEL_KEYS here, we'll use keys_found later

# Gemma-12B
# Check if 'results_gemma12b_json' variable exists and is a non-empty list
if 'results_gemma12b_json' in locals() and isinstance(results_gemma12b_json, list) and results_gemma12b_json:
    final_model_results_lists['gemma_12b'] = results_gemma12b_json
    keys_found.append('gemma_12b')
    print(f"  [OK] Found {len(results_gemma12b_json)} results for gemma_12b.")
else:
    print(f"  [WARNING] Results for gemma_12b ('results_gemma12b_json') not found or empty. It will be excluded.")

# Mistral-Small (2501 version)
# Check if 'results_mistral_small_json' variable exists and is a non-empty list
if 'results_mistral_small_json' in locals() and isinstance(results_mistral_small_json, list) and results_mistral_small_json:
    final_model_results_lists['mistral_small'] = results_mistral_small_json
    keys_found.append('mistral_small')
    print(f"  [OK] Found {len(results_mistral_small_json)} results for mistral_small.")
else:
    print(f"  [WARNING] Results for mistral_small ('results_mistral_small_json') not found or empty. It will be excluded.")

# --- Final Check ---
# Use the list of keys actually found
if len(keys_found) < 2:
    # Need at least 2 models for any meaningful comparison/voting
    raise SystemExit(f"ERROR: Found valid results for only {len(keys_found)} model(s) ({keys_found}). Need at least 2 to proceed with merging and voting.")
elif len(keys_found) < 3:
    # This condition might be hit if one model's results were missing
    print(f"\n[WARNING] Proceeding with only {len(keys_found)} models ({keys_found}). Majority vote might be less robust (ties possible if only 2 models).")
else:
    # This means results for all 3 intended models were found
    print(f"\nSuccessfully gathered results for all {len(keys_found)} models: {keys_found}. Ready for merging.")

# Ensure filtered_df exists for merging context later
if 'filtered_df' not in locals():
     raise NameError("ERROR: Original 'filtered_df' not found. Needed for context merging later.")
elif not isinstance(filtered_df, pd.DataFrame) or filtered_df.empty:
     raise ValueError("ERROR: 'filtered_df' exists but is not a valid, non-empty DataFrame.")
else:
    print(f"Original 'filtered_df' found with shape {filtered_df.shape}.")


print("="*60)
print(f"Models to be used in next steps: {keys_found}") # Explicitly state which models will proceed

In [ ]:
# Cell: Merge Model Results

print("\n" + "="*30 + " Step 2: Merging Model Results " + "="*30)

df_merged_all_final = None

# Iterate through the validated model results
for i, model_key in enumerate(FINAL_MODEL_KEYS):
    results_list = final_model_results_lists[model_key]
    print(f"Processing results for: {model_key}")
    try:
        # Create DataFrame from the list of dictionaries
        df_model = pd.DataFrame(results_list)

        # --- Data Validation ---
        # Ensure essential columns exist
        required_cols = ['query_id', 'product_id', 'accurate_label', 'reformulated_query', 'reasoning', 'parsing_error']
        if not all(col in df_model.columns for col in required_cols):
             missing = [col for col in required_cols if col not in df_model.columns]
             print(f"  [ERROR] Missing required columns for {model_key}: {missing}. Skipping this model.")
             # Remove the problematic key from our list
             FINAL_MODEL_KEYS.remove(model_key)
             final_model_results_lists.pop(model_key, None) # Remove from dict too
             continue # Skip to next model

        # Select and rename columns to be model-specific
        # Using f-strings for cleaner renaming
        df_model_renamed = df_model[required_cols].rename(columns={
            'accurate_label': f'acc_{model_key}',
            'reformulated_query': f'reform_{model_key}',
            'reasoning': f'reason_{model_key}',
            'parsing_error': f'parse_err_{model_key}'
        })

        # --- Merge ---
        if df_merged_all_final is None:
            # Initialize with the first valid model's results
            df_merged_all_final = df_model_renamed
            print(f"  Initialized merged DataFrame with {model_key} results.")
        else:
            # Merge subsequent models using an outer join
            # Outer join ensures rows are kept even if one model failed on that specific query/product pair
            df_merged_all_final = pd.merge(
                df_merged_all_final,
                df_model_renamed,
                on=['query_id', 'product_id'],
                how='outer'
            )
            print(f"  Merged results from {model_key}.")

    except Exception as e:
        print(f"  [ERROR] Failed to process or merge results for model {model_key}: {e}")
        # Remove the problematic key if an error occurs during processing
        if model_key in FINAL_MODEL_KEYS: FINAL_MODEL_KEYS.remove(model_key)
        final_model_results_lists.pop(model_key, None)


# --- Final Check on Merged DataFrame ---
if df_merged_all_final is None:
    raise SystemExit("ERROR: Failed to create a merged DataFrame from any model results.")
elif len(FINAL_MODEL_KEYS) < 2:
     raise SystemExit(f"ERROR: After merging, only {len(FINAL_MODEL_KEYS)} model(s) remain. Need at least 2.")
else:
    print(f"\nSuccessfully merged results from {len(FINAL_MODEL_KEYS)} models into df_merged_all_final.")
    print("--- Preview of Merged DataFrame (First 5 Rows) ---")
    display(df_merged_all_final.head())

print("="*60)

In [ ]:
# Cell: Define Final Majority Voting Function

print("\n" + "="*30 + " Step 3: Defining Majority Voting Logic " + "="*30)

def get_majority_vote_final(row, model_keys):
    """
    Performs majority voting based on results from multiple LLMs for a single item.

    Determines the final accuracy label (True, False, or None for tie/error)
    and the consensus reformulated query if the final label is False.

    Parameters:
        row (pd.Series): A row from the merged DataFrame containing results
                         (acc_*, reform_*, reason_*, parse_err_*) for each model key.
        model_keys (list): A list of model keys participating in the vote
                           (e.g., ['qwen7b', 'gemma12b', 'mistral_small']).

    Returns:
        pd.Series: Contains the final voted label ('final_accurate'), aggregated
                   reasoning ('consensus_reasoning'), the consensus reformulated
                   query ('final_reformulated'), and any voting/parsing errors
                   encountered ('voting_errors').
    """
    votes = [] # List to store valid boolean votes (True/False)
    reformulations_if_false = [] # Collect reformulations ONLY from models that voted False
    reasonings = [] # Collect reasoning strings from all valid votes
    parsing_errors = [] # Collect errors encountered for this row

    # --- Iterate through each model's results for the current row ---
    for key in model_keys:
        # Define column names for this model
        acc_col = f'acc_{key}'
        reason_col = f'reason_{key}'
        reform_col = f'reform_{key}'
        parse_err_col = f'parse_err_{key}'

        # --- Robustness Checks ---
        # 1. Check if columns for this model exist in the row (handle missing merge data)
        if acc_col not in row.index or parse_err_col not in row.index:
             parsing_errors.append(f"{key}: Result columns missing in merged data.")
             continue # Skip to the next model for this row

        # 2. Check if the model had a parsing error for this row
        if pd.notna(row.get(parse_err_col)):
            parsing_errors.append(f"{key}: Parsing Failed ({row.get(parse_err_col)})")
            continue # Skip vote from this model as it failed parsing

        # 3. Check if the accuracy vote is a valid boolean value
        vote_val = row.get(acc_col)
        if not isinstance(vote_val, bool): # Includes check for None/NaN
            parsing_errors.append(f"{key}: Invalid 'accurate_label' value ({vote_val}). Expected boolean.")
            continue # Skip vote from this model

        # --- If checks pass, record the valid vote and reasoning ---
        votes.append(vote_val)
        reasonings.append(f"[{key.upper()} Vote: {vote_val}] {row.get(reason_col, 'N/A')}") # Add reasoning

        # 4. If the vote was False, record its proposed reformulation (if any)
        if not vote_val: # If vote is False
            reformulation = row.get(reform_col)
            if pd.notna(reformulation) and isinstance(reformulation, str) and reformulation.strip():
                reformulations_if_false.append(reformulation)
            # else: # Optional: note if a False voter didn't provide reformulation
            #     reasonings.append(f"[{key.upper()} Note: Voted False but no reformulation provided.]")

    # --- Determine the Final Verdict based on collected valid votes ---
    num_valid_votes = len(votes)
    final_accurate = None # Default to None (tie or error)
    final_reformulated = None # Default to None

    if num_valid_votes == 0:
        consensus_reasoning = "No valid votes received from any model."
        # final_accurate remains None
    else:
        num_true = sum(votes) # Count True votes
        num_false = num_valid_votes - num_true # Count False votes

        # Apply majority rule
        if num_true > num_false:
            final_accurate = True
        elif num_false > num_true:
            final_accurate = False
        else: # Tie (e.g., 1 True, 1 False if only 2 models worked)
            final_accurate = None # Explicitly mark ties as None/Undecided
            reasonings.append("[Result: Tied Vote or Insufficient Votes]")

        # Combine all collected reasoning strings
        consensus_reasoning = "; ".join(reasonings)

        # Determine the consensus reformulation ONLY IF the final verdict is False
        if final_accurate is False and reformulations_if_false:
            # Find the most common reformulation proposed by the models that voted False
            reform_counts = Counter(reformulations_if_false)
            if reform_counts: # Ensure the counter is not empty
                # Get the most frequent reformulation
                final_reformulated = reform_counts.most_common(1)[0][0]
            # else: # This case shouldn't happen if reformulations_if_false is not empty
            #    final_reformulated = "Error: No reformulations found despite False votes."

    # --- Compile Voting Errors ---
    voting_errors_str = "; ".join(parsing_errors) if parsing_errors else None # Use None if no errors

    # --- Return results as a Pandas Series ---
    return pd.Series([final_accurate, consensus_reasoning, final_reformulated, voting_errors_str],
                     index=['final_accurate', 'consensus_reasoning', 'final_reformulated', 'voting_errors'])

print("Majority voting function 'get_majority_vote_final' defined.")
print("It handles multiple models, checks for valid votes, determines consensus,")
print("and selects the most common reformulation if the final vote is False.")
print("="*60)

In [ ]:
# Cell: Apply Voting and Create Final Output DataFrame

print("\n" + "="*30 + " Step 4: Applying Voting & Creating Final Output " + "="*30)

# --- Apply the voting function ---
# Ensure we pass the final list of model keys that were successfully merged
print(f"Applying majority vote using models: {FINAL_MODEL_KEYS}...")

if df_merged_all_final is None:
     raise ValueError("Merged DataFrame 'df_merged_all_final' is not available.")

# Apply the function row-wise (axis=1)
# This returns a DataFrame where each row contains the Series from get_majority_vote_final
vote_results_final = df_merged_all_final.apply(
    lambda row: get_majority_vote_final(row, FINAL_MODEL_KEYS),
    axis=1
)

print("Voting function applied.")

# --- Combine vote results with identifiers ---
# Concatenate the original identifiers with the new vote results
df_voted_with_ids = pd.concat([df_merged_all_final[['query_id', 'product_id']], vote_results_final], axis=1)

# --- Merge back the original query text for context ---
# Ensure filtered_df is available and has the required columns
if 'filtered_df' not in locals() or not all(c in filtered_df.columns for c in ['query_id', 'product_id', 'query']):
    print("[WARNING] Original 'filtered_df' or required columns missing. Cannot merge back query text.")
    df_final_output_table = df_voted_with_ids # Proceed without the query text
else:
    print("Merging back original query text...")
    # Use a left merge to keep all voted rows and add the query
    df_final_output_table = pd.merge(
        df_voted_with_ids,
        filtered_df[['query_id', 'product_id', 'query']].drop_duplicates(), # Avoid duplicate queries if any
        on=['query_id', 'product_id'],
        how='left'
    )

# --- Select and Rename Columns for Final Output ---
# Define the columns required by the exercise specification
required_output_cols = ['query_id', 'product_id', 'final_accurate', 'final_reformulated']
# Define optional columns for analysis/review
analysis_cols = ['query', 'consensus_reasoning', 'voting_errors']

# Ensure all required columns exist after merge/concat
if not all(c in df_final_output_table.columns for c in required_output_cols):
    missing_final = [c for c in required_output_cols if c not in df_final_output_table.columns]
    raise ValueError(f"Missing essential columns in final table: {missing_final}")

# Select columns in desired order and rename
df_final_output_table = df_final_output_table[
    ['query_id', 'product_id'] + # Identifiers
    analysis_cols +              # Context/Analysis columns (optional but useful here)
    ['final_accurate', 'final_reformulated', 'voting_errors'] # Vote results
].rename(columns={
    'final_accurate': 'accurate_label',     # Rename to match exercise requirement
    'final_reformulated': 'reformulated_query' # Rename to match exercise requirement
})


print("\n--- Final Output Table (First 10 Rows) ---")
# Display the final table, including reasoning and errors for review
with pd.option_context('display.max_rows', 10, 'display.max_columns', None, 'display.width', 1000):
    display(df_final_output_table.head(25))

print("="*60)

In [ ]:
# Cell: Save Final Results to CSV

print("\n" + "="*30 + " Step 5: Saving Final Results " + "="*30)

# Define the filename for the final submission CSV
output_filename_final = "grainger_llm_verification_results_final_vote_04_22.csv"

# Define the columns required for the final submission file
submission_columns = ['query_id', 'product_id', 'accurate_label', 'reformulated_query']

# --- Create the submission DataFrame ---
try:
    if not all(col in df_final_output_table.columns for col in submission_columns):
        missing_sub = [col for col in submission_columns if col not in df_final_output_table.columns]
        raise ValueError(f"Cannot create submission file. Missing required columns: {missing_sub}")

    submission_df = df_final_output_table[submission_columns].copy()

    # --- Handle potential non-standard data types before saving ---
    # Convert boolean accurate_label to string 'True'/'False' or keep as boolean?
    # Let's keep as boolean for now, CSV usually handles it. Nulls will be empty.
    # Ensure reformulated_query is string or empty string (replace None/NaN)
    submission_df['reformulated_query'] = submission_df['reformulated_query'].fillna('').astype(str)

    # --- Save to CSV ---
    submission_df.to_csv(output_filename_final, index=False, encoding='utf-8')
    print(f"Successfully saved final submission file with required columns to:")
    print(f"'{output_filename_final}'")

    # --- Optional: Save the full table with reasoning for review ---
    output_filename_full = "grainger_llm_verification_results_final_vote_FULL.csv"
    df_final_output_table.to_csv(output_filename_full, index=False, encoding='utf-8')
    print(f"\nOptionally saved full results (including reasoning) to:")
    print(f"'{output_filename_full}'")

except Exception as e:
    print(f"\n[ERROR] Failed to save final results to CSV: {e}")

print("="*60)

In [ ]:
# Cell: Analyze Results - Overall Statistics (Corrected)

print("\n" + "="*30 + " Step : Final Result Statistics " + "="*30)

if 'df_final_output_table' not in locals():
    print("[ERROR] Final output table 'df_final_output_table' not found. Cannot perform analysis.")
else:
    total_rows_final = len(df_final_output_table)
    print(f"Analysis based on final vote from models: {FINAL_MODEL_KEYS}") # Ensure FINAL_MODEL_KEYS is defined from Cell 1
    print(f"Total Rows Analyzed: {total_rows_final}")

    # --- Voting Errors ---
    # Recalculate the count ensuring it's a scalar integer
    # .notna() creates a boolean Series (True where not null)
    # .sum() on a boolean Series treats True as 1, False as 0, and sums them
    voting_errors_scalar_count = df_final_output_table['voting_errors'].notna().sum()

    # Ensure it's an integer (it should be, but belt-and-suspenders)
    if isinstance(voting_errors_scalar_count, pd.Series):
         # If it's still a Series, sum the Series values to get the total count
         print("[Warning] Initial sum resulted in a Series, summing Series values.")
         voting_errors_scalar_count = voting_errors_scalar_count.sum()
    else:
         # Cast to int just to be certain for the comparison
         voting_errors_scalar_count = int(voting_errors_scalar_count)

    print(f"\nRows with Voting/Parsing Errors Encountered: {voting_errors_scalar_count}") # Print the scalar count

    # Now the 'if' statement compares an integer to an integer
    if voting_errors_scalar_count > 0:
        print("  (These rows might have resulted in a 'Tied/None' final label if no valid votes were cast)")
        # Optional: Display rows with errors
        # print("\n--- Rows with Voting/Parsing Errors ---")
        # with pd.option_context('display.max_rows', 10):
        #    display(df_final_output_table[df_final_output_table['voting_errors'].notna()])

    # --- Final Label Distribution ---
    # Use value_counts, including NaN (which represents ties/errors)
    final_label_counts = df_final_output_table['accurate_label'].value_counts(dropna=False).rename({
        True: 'Accurate',      # Label True as 'Accurate'
        False: 'Inaccurate',    # Label False as 'Inaccurate'
        np.nan: 'Tied/Undecided' # Label NaN/None as 'Tied/Undecided'
    })

    print("\n--- Final Label Distribution (Majority Vote) ---")
    if not final_label_counts.empty:
        print(final_label_counts)
    else:
        print("No labels to count.")

    # --- Calculate Percentages ---
    print("\n--- Percentage Distribution ---")
    if total_rows_final > 0 and not final_label_counts.empty:
        for label, count in final_label_counts.items():
            percentage = (count / total_rows_final) * 100
            print(f"  {label}: {count} ({percentage:.1f}%)")
    else:
        print("  Cannot calculate percentages (0 rows or no labels).")

print("="*60)

In [ ]:
# Cell: Analyze Results - Tied/Undecided Cases (Modified to include Product Context)

print("\n" + "="*30 + " Step : Analysis of Tied/Undecided Cases " + "="*30)

if 'df_final_output_table' not in locals() or 'df_merged_all_final' not in locals() or 'filtered_df' not in locals():
    print("[ERROR] Required DataFrames ('df_final_output_table', 'df_merged_all_final', 'filtered_df') not found.")
else:
    # Filter for rows where the final accurate_label is Null/NaN
    df_tied_final = df_final_output_table[df_final_output_table['accurate_label'].isnull()].copy()
    num_tied_final = len(df_tied_final)

    print(f"Number of items where vote was Tied or Undecided: {num_tied_final}")

    if num_tied_final > 0:
        print("\n--- Details of Tied/Undecided Cases ---")
        print("Review query, product context, individual model votes, and reasoning below.")

        # 1. Merge final tied results with individual model votes/reasons
        df_tied_details_merged = pd.merge(
            df_tied_final[['query_id', 'product_id', 'query', 'consensus_reasoning', 'voting_errors']], # Start with key info from final table
            df_merged_all_final,                                # Merge with the table holding individual model details
            on=['query_id', 'product_id'],
            how='left'
        )

        # 2. Merge with original filtered data to get product context
        df_tied_details_final_with_context = pd.merge(
            df_tied_details_merged,
            filtered_df[['query_id', 'product_id', 'llm_product_context']].drop_duplicates(), # Get product context
            on=['query_id', 'product_id'],
            how='left'
        )

        # Define columns to display for clarity
        display_cols_tied = [
            'query_id', 'product_id', 'query',
            'llm_product_context', # <-- Added product context
            'consensus_reasoning', 'voting_errors'
        ]
        # Add individual model accuracy, reasoning, and parsing error columns
        for key in FINAL_MODEL_KEYS: # Use the globally defined FINAL_MODEL_KEYS
            display_cols_tied.extend([f'acc_{key}', f'reason_{key}', f'parse_err_{key}'])

        # Ensure the columns exist before trying to display
        display_cols_tied = [col for col in display_cols_tied if col in df_tied_details_final_with_context.columns]

        # Adjust display width to accommodate the context
        with pd.option_context('display.max_colwidth', 250, 'display.max_rows', 50, 'display.width', 1500):
             display(df_tied_details_final_with_context[display_cols_tied])

        print("\nPossible reasons for Tied/Undecided:")
        print("  - Equal number of True/False votes.")
        print("  - No valid votes received due to parsing errors.")
        print("  - Ambiguity in product description vs. query specifications.")
    else:
        print("No items had a Tied/Undecided final vote.")

print("="*60)

In [ ]:
# Cell: Analyze Results - Consensus Inaccurate Cases

print("\n" + "="*30 + " Step : Analysis of Consensus Inaccurate Cases " + "="*30)

if 'df_final_output_table' not in locals() or 'df_merged_all_final' not in locals() or 'filtered_df' not in locals():
    print("[ERROR] Required DataFrames ('df_final_output_table', 'df_merged_all_final', 'filtered_df') not found.")
else:
    # Filter for rows where the final accurate_label is False
    df_inaccurate_final = df_final_output_table[df_final_output_table['accurate_label'] == False].copy()
    num_inaccurate_final = len(df_inaccurate_final)

    print(f"Number of items where consensus vote was 'Inaccurate': {num_inaccurate_final}")

    if num_inaccurate_final > 0:
        print("\n--- Details of Consensus Inaccurate Cases ---")
        print("Review query vs product context, the consensus reasoning, and the proposed reformulated query.")

        # 1. Merge final inaccurate results with individual model accuracy votes
        df_inaccurate_merged_acc = pd.merge(
            df_inaccurate_final[['query_id', 'product_id', 'query', 'reformulated_query', 'consensus_reasoning']],
            # Select only query_id, product_id and acc_* columns from the wide merge table
            df_merged_all_final[['query_id', 'product_id'] + [f'acc_{key}' for key in FINAL_MODEL_KEYS if f'acc_{key}' in df_merged_all_final.columns]],
            on=['query_id', 'product_id'],
            how='left'
        )

        # 2. Merge with original filtered data to get product context
        df_inaccurate_details_final_with_context = pd.merge(
            df_inaccurate_merged_acc,
            filtered_df[['query_id', 'product_id', 'llm_product_context']].drop_duplicates(), # Get product context
            on=['query_id', 'product_id'],
            how='left'
        )


        # Define columns to display
        display_cols_inacc = [
            'query_id', 'product_id', 'query',
            'llm_product_context', # <-- product context
            'reformulated_query', 'consensus_reasoning'
        ]
        # Add individual model accuracy columns to see the votes
        for key in FINAL_MODEL_KEYS:
            acc_col = f'acc_{key}'
            if acc_col in df_inaccurate_details_final_with_context.columns:
                display_cols_inacc.append(acc_col)

        # Adjust display width
        with pd.option_context('display.max_colwidth', 1050, 'display.max_rows', 50, 'display.width', 1500):
             display(df_inaccurate_details_final_with_context[display_cols_inacc])
    else:
        print("No items had a consensus 'Inaccurate' final vote.")

print("="*60)

In [ ]:
# Cell: Analyze Results - Consensus Accurate Cases

print("\n" + "="*30 + " Step 9: Analysis of Consensus Accurate Cases " + "="*30)

if 'df_final_output_table' not in locals() or 'filtered_df' not in locals():
    print("[ERROR] Required DataFrames ('df_final_output_table', 'filtered_df') not found.")
else:
    # Filter for rows where the final accurate_label is True
    df_accurate_final = df_final_output_table[df_final_output_table['accurate_label'] == True].copy()
    num_accurate_final = len(df_accurate_final)

    print(f"Number of items where consensus vote was 'Accurate': {num_accurate_final}")

    if num_accurate_final > 0:
        print("\n--- Details of Consensus Accurate Cases (Sample) ---")
        print("Spot-check query vs product context and reasoning to ensure rules were applied correctly.")

        # Merge accurate results with original filtered data to get product context
        df_accurate_details_final_with_context = pd.merge(
            df_accurate_final[['query_id', 'product_id', 'query', 'consensus_reasoning']], # Select key columns
            filtered_df[['query_id', 'product_id', 'llm_product_context']].drop_duplicates(), # Get product context
            on=['query_id', 'product_id'],
            how='left'
        )

        # Define columns to display
        display_cols_acc = [
            'query_id', 'product_id', 'query',
            'llm_product_context', # <--  product context
            'consensus_reasoning'
        ]
        num_to_show = min(10, num_accurate_final) # Show up to 10 examples

        # Adjust display width
        with pd.option_context('display.max_colwidth', 250, 'display.max_rows', 10, 'display.width', 1500):
             display(df_accurate_details_final_with_context.head(num_to_show)[display_cols_acc])
    else:
        print("No items had a consensus 'Accurate' final vote.")

print("="*60)

In [ ]:
# Cell: Generate Executive Summary (Reflecting Final 3-Model Results)

from IPython.display import display, Markdown
import warnings # Import warnings if not already done

# Ensure config is available if using config paths for filenames
try:
    from src import config # Assuming config.py is in src directory accessible via path
except ImportError:
    config = None # Set config to None if it can't be imported
    warnings.warn("Could not import config.py. Filenames in summary might use defaults.")


print("\n" + "="*30 + " Step 10: Executive Summary Generation " + "="*30)

# --- Verify required variables exist ---

required_data_vars = {
    'df_final_output_table': 'df_final_output_table', # Check this name carefully
    'final_label_counts': 'final_label_counts',      # Check this name carefully
    'FINAL_MODEL_KEYS': 'FINAL_MODEL_KEYS',         # Should be ['qwen_14b', 'gemma_12b', 'mistral_small']
    'voting_errors_scalar_count': 'voting_errors_scalar_count', # Should be 0
    'num_inaccurate_final': 'num_inaccurate_final'         # Should be
}

missing_vars = []
for display_name, actual_name in required_data_vars.items():
    if actual_name not in locals():
        missing_vars.append(f"'{actual_name}' (needed for {display_name})")

# --- Proceed only if essential variables exist ---
if missing_vars:
     summary_md = f"## LLM Verification Summary\n\nERROR: Could not generate summary. Essential analysis data missing:\n- {chr(10).join(missing_vars)}"
     warnings.warn(f"Missing variables needed for summary: {', '.join(missing_vars)}")
     display(Markdown(summary_md)) # Display the error message
else:
    # --- Prepare data for summary using variables verified to exist ---
    df_final_output_table = locals()[required_data_vars['df_final_output_table']]
    final_label_counts = locals()[required_data_vars['final_label_counts']]
    FINAL_MODEL_KEYS = locals()[required_data_vars['FINAL_MODEL_KEYS']]
    voting_errors_count = locals()[required_data_vars['voting_errors_scalar_count']] # Should be 0
    num_inaccurate = locals()[required_data_vars['num_inaccurate_final']]           # Should be 8

    # Retrieve counts safely using .get() on the final_label_counts Series
    # Use the RENAMED labels
    num_accurate = final_label_counts.get('Accurate', 0)       # Should be 16
    # This should be 0 since only Accurate/Inaccurate were in final_label_counts
    num_tied_undecided = final_label_counts.get('Tied/Undecided', 0)

    total_items = len(df_final_output_table) # Should be 24

    # Helper function to calculate and format percentage string safely
    def format_percentage_string(count, total):
        """Calculates percentage and formats to string 'XX.X%', or returns 'N/A'."""
        if isinstance(count, int) and isinstance(total, int) and total > 0:
            percentage_value = (count / total)
            return f"{percentage_value:.1%}"
        else:
            return "N/A"

    # Calculate percentages using the helper
    perc_accurate = format_percentage_string(num_accurate, total_items)         # Should be 66.7%
    perc_inaccurate = format_percentage_string(num_inaccurate, total_items)     # Should be 33.3%
    perc_tied_undecided = format_percentage_string(num_tied_undecided, total_items) # Should be 0.0%

    # Get final filenames (ensure these variables exist from saving step, or use config)
    # Provide default values if config or specific variables aren't found
    default_final_csv = 'results/grainger_llm_verification_results_final_vote.csv'
    default_full_csv = 'results/grainger_llm_verification_results_final_vote_FULL.csv'

    final_csv = default_final_csv
    full_csv = default_full_csv

    if config and hasattr(config, 'FINAL_OUTPUT_CSV'):
        final_csv = config.FINAL_OUTPUT_CSV
    if config and hasattr(config, 'FULL_OUTPUT_CSV'):
         full_csv = config.FULL_OUTPUT_CSV

    # Override with explicit variables if they exist from a saving cell
    if 'output_filename_final' in locals(): final_csv = output_filename_final
    if 'output_filename_full' in locals(): full_csv = output_filename_full

    # --- Construct Markdown Summary ---
    summary_md = f"""
## LLM Verification Task: Executive Summary

**Objective:** Verify the accuracy of the 'Exact' (E) label for **{total_items}** specific query-product pairs using a multi-LLM approach (**{', '.join(FINAL_MODEL_KEYS)}**) and majority voting. Reformulate queries where the 'E' label was deemed inaccurate due to explicit contradictions.

**Methodology:**
1.  **Data Preparation:** Loaded ESCI data, merged product details, filtered for 3 target queries with 'E' labels (`aa batteries 100 pack`, `kodak photo paper...`, `dewalt 8v...`).
2.  **LLM Inference:** Used three instruction-tuned LLMs ({', '.join(FINAL_MODEL_KEYS)}) with 4-bit quantization for memory efficiency. Employed detailed prompts enforcing strict rules (contradiction vs. missing info) and required JSON output.
3.  **Aggregation:** Applied majority voting (2 out of 3 model agreement required for consensus) to determine the final `accurate_label`. Selected the most common `reformulated_query` when the consensus was 'Inaccurate'.

**Key Findings & Metrics:**

*   **Total Items Analyzed:** {total_items}
*   **Final Label Distribution (Consensus Vote):**
    *   ✅ **Accurate:** **{num_accurate}** ({perc_accurate}) - *Original 'E' label confirmed correct by model majority.*
    *   ❌ **Inaccurate:** **{num_inaccurate}** ({perc_inaccurate}) - *Original 'E' label deemed incorrect by model majority due to contradictions.*
    *   ❓ **Tied / Undecided:** **{num_tied_undecided}** ({perc_tied_undecided}) - *No majority reached (Resulted in 0 cases with 3 models).*
*   **Items with Processing Errors during Vote:** {voting_errors_count} (Resulted in 0 cases).

**Analysis Highlights:**

1.  **Inaccurate Labels Identified:** The **{num_inaccurate}** items marked 'Inaccurate' represent clear instances where product details explicitly contradicted the search query. Common contradiction types observed included:
    *   **Pack Size Mismatch:** (e.g., '50 count' or '60 count' vs. query '100 pack')
    *   **Attribute Mismatch:** (e.g., 'matte' vs. query 'glossy')
    *   **Specification Mismatch:** (e.g., '12V MAX' vs. query '8V MAX')
    *   **Brand Mismatch:** (e.g., 'ENERTWIST'/'Doaaler' vs. query 'DEWALT'/'Kodak')
    *   **Product Type Mismatch:** (e.g., 'Battery Charger' vs. query 'Screwdriver Kit')
    The generated `reformulated_query` in the output file provides suggested corrections based on the actual product data.

2.  **Consensus Reached:** With three models providing valid outputs, a majority consensus (at least 2/3 agreement) was achieved for **all {total_items} items** ({num_accurate + num_inaccurate}/{total_items} = 100%), resulting in **0 'Tied / Undecided'** cases. This demonstrates the decisiveness of the 3-model ensemble.

3.  **Model Agreement & Robustness:** The ensemble approach, using {len(FINAL_MODEL_KEYS)} diverse models, provided robust verification. The 100% consensus rate indicates consistent interpretation of the rules and product information across the models for these specific examples.

**Recommendations:**

*   **Action Inaccurate Findings:** Prioritize reviewing the **{num_inaccurate}** items flagged as 'Inaccurate'. Evaluate the LLM-generated `reformulated_query` (available in `{final_csv}`) and consider using this information for potential catalog updates or search relevance tuning.
*   **Leverage High Confidence Results:** The **{num_accurate}** items confirmed as 'Accurate' with unanimous (3/3) or strong majority (2/3) consensus can potentially bypass some levels of manual review, optimizing resource allocation.
*   **Future Considerations:** While effective for this subset, scaling would require infrastructure optimizations (batching, parallel processing). For subtler ambiguities not seen here, prompt refinement or incorporating structured data extraction could be explored. Consider evaluating the cost/benefit of using 3 models vs. perhaps a single, highly capable model if API costs were a factor in production.

**Output:** The final results table required by the exercise (containing `query_id`, `product_id`, `accurate_label`, `reformulated_query`) is saved in `{final_csv}`. A full table including detailed consensus reasoning and individual model votes is available in `{full_csv}` for deeper analysis.
"""

# --- Display the Summary ---
display(Markdown(summary_md))
print("="*60)

## Conclusion: Superiority and Relevance for Grainger

For tasks requiring high accuracy and reliability in interpreting complex rules against potentially noisy data like the one in our dataset with missing descriptions et  – precisely the challenge faced when ensuring customers find the *exact* MRO products they need – the multi-model ensemble approach is technically superior to relying on a single LLM. Its benefits directly align with Grainger's business objectives and operational needs:

1.  **Higher Reliability for Catalog Integrity:** Grainger's success relies on providing accurate product information. The ensemble method significantly reduces the risk of single-model errors or biases leading to incorrect verification of 'Exact' matches. This enhanced reliability is crucial for maintaining the integrity of Grainger's vast MRO catalog, ensuring customers receive products that meet their stated specifications and fostering trust.

2.  **Increased Confidence in Search & Customer Experience:** By providing a more trustworthy verification of 'E' labels, this approach directly improves the quality and relevance of search results. When a customer searches for a specific item (like the "dewalt 8v max... gyroscopic" screwdriver), they can have higher confidence that an 'Exact' match truly meets all criteria, leading to a more efficient and successful purchasing experience, minimizing frustration and returns.

3.  **Built-in Ambiguity Detection for Complex MRO Data:** MRO products often have highly specific, technical attributes where misinterpretation can be costly. The ensemble's ability to flag 'Tied/Undecided' cases is invaluable. It identifies specific query-product pairs where descriptions might be ambiguous or rules are hard to apply, allowing Grainger's product experts to focus their manual review efforts precisely where automated consensus couldn't be reached, preventing potential errors for customers needing specific parts.

4.  **Practicality and Scalability Validation:** Demonstrating that powerful, recent open-source models can be effectively run locally via quantization validates a practical path forward. While Grainger's full catalog is immense, this approach shows feasibility for targeted quality checks, validating new product data, or analyzing problematic search segments without immediate, large-scale reliance on potentially costly external APIs for every check, aligning with Grainger's history of pragmatic technology adoption.

In essence, while computationally more intensive than a single LLM call, the significant gains in **robustness, trustworthiness, and the ability to intelligently flag ambiguity** justify the ensemble approach for a critical data quality task that underpins Grainger's core value proposition: helping customers confidently find the *right* products. It moves beyond a simple LLM proof-of-concept towards a more engineered, reliable AI system suited for enhancing Grainger's operational excellence and customer satisfaction.